## Loading data

In [ ]:
# !pip install pydicom nibabel albumentations --upgrade

In [ ]:
!pip install -U monai

In [ ]:
import warnings
warnings.filterwarnings("ignore")

!nvidia-smi
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob, natsort
import pandas as pd

In [ ]:
# axis method
# 1. PCA (To do)
# 2. Minimum Rotational inertia
# 3. Ellipse
# !pip install kornia albumentations

In [ ]:
from skimage.measure import label, regionprops, regionprops_table
import math, kornia

# # inertia axis
# def visualize(**images):
#     """Plot images in one row."""
#     angles = list()
    
#     n = len(images)
#     plt.figure(figsize=(32, 16))
#     for i, (name, image) in enumerate(images.items()):
#         if i==0 and image.shape[0] == 3:
#             image_main = image.permute(1,2,0).numpy()#.int()
#         if torch.is_tensor(image) and image.shape[0] == 3:
#             image = image.permute(1,2,0).numpy()#.int()
#         else:
#             try:
#                 image = image.numpy()
#             except:
#                 pass
            
#         plt.subplot(1, n, i + 1)
#         plt.xticks([])
#         plt.yticks([])
#         plt.title(' '.join(name.split('_')).title())
#         plt.imshow(image_main,cmap='gray')
#         if i>=1:
#             prop = inertia_axis(image)
#             coord_x = int(prop[0])
#             coord_y = int(prop[1])
#             angle = prop[2]
#             offset= 200
#             image[coord_y-5:coord_y+5,coord_x-5:coord_x+5] = 0
#             image = cv2.line(image,(coord_x-offset, coord_y-int(offset*np.tan(angle))),(coord_x+offset, coord_y+int(offset*np.tan(angle))),(0,255,255),5)
#             angle = np.degrees(angle)*(-1)
#             angles.append(angle)
#             plt.title(' '.join(name.split('_')).title()+'_'+'{:.2f}'.format(angle))
#             plt.imshow(image,cmap='gray',alpha=0.4)
#             plt.plot(coord_x, coord_y, '.g', markersize=15)
#     plt.show()
#     return angles

# # ellipse
# def visualize(**images):
#     """Plot images in one row."""
#     angles = list()
    
#     n = len(images)
#     plt.figure(figsize=(32, 16))
#     for i, (name, image) in enumerate(images.items()):
#         if i==0 and image.shape[0] == 3:
#             image_main = image.permute(1,2,0).numpy()#.int()
#         if torch.is_tensor(image) and image.shape[0] == 3:
#             image = image.permute(1,2,0).numpy()#.int()
#         else:
#             try:
#                 image = image.numpy()
#             except:
#                 pass
            
#         plt.subplot(1, n, i + 1)
#         plt.xticks([])
#         plt.yticks([])
#         plt.title(' '.join(name.split('_')).title())
#         plt.imshow(image_main,cmap='gray')
#         if i>=1:            
#             image = image.astype(np.uint8)
#             regions = regionprops(image)
#             for props in regions:
#                 y0, x0 = props.centroid
#                 orientation = props.orientation
#                 x1 = x0 + math.sin(orientation) * 0.5 * props.major_axis_length
#                 y1 = y0 + math.cos(orientation) * 0.5 * props.major_axis_length
#                 x2 = x0 - math.sin(orientation) * 0.5 * props.major_axis_length
#                 y2 = y0 - math.cos(orientation) * 0.5 * props.major_axis_length
#                 x3 = x0 + math.cos(orientation) * 0.5 * props.minor_axis_length
#                 y3 = y0 - math.sin(orientation) * 0.5 * props.minor_axis_length
#                 x4 = x0 - math.cos(orientation) * 0.5 * props.minor_axis_length
#                 y4 = y0 + math.sin(orientation) * 0.5 * props.minor_axis_length
#                 if i<5:
#                     angle = - np.tan((y0 - y2) / (x0 - x2))
#                     plt.plot((x0, x1), (y0, y1), 'black', linewidth=1)
#                     plt.plot((x0, x2), (y0, y2), 'black', linewidth=1)
#                     plt.plot(x0, y0, '.g', markersize=15)
#                 else:
#                     angle = - np.tan((y0 - y3) / (x0 - x3))
#                     plt.plot((x0, x3), (y0, y3), 'black', linewidth=1)
#                     plt.plot((x0, x4), (y0, y4), 'black', linewidth=1)
#                     plt.plot(x0, y0, '.g', markersize=15)
                    
#                 if angle >= np.pi:
#                     angle = angle - np.pi
#                 elif angle <= - np.pi:
#                     angle = np.pi + angle 
#                 angle = np.degrees(angle)
# #             image[coord_y-5:coord_y+5,coord_x-5:coord_x+5] = 0
# #             image = cv2.line(image,(coord_x-offset, coord_y-int(offset*np.tan(angle))),(coord_x+offset, coord_y+int(offset*np.tan(angle))),(0,255,255),5)
# #             angle = np.degrees(angle)*(-1)
#                 angles.append(angle)
# #             angle=0
#             try:
#                 plt.title(' '.join(name.split('_')).title()+'_'+'{:.2f}'.format(angle))
#             except:
#                 plt.title(' '.join(name.split('_')).title()+'_')                
#             plt.imshow(image,cmap='gray',alpha=0.4)
#     plt.show()
#     return angles

#################### upper is old ##################################### 0

# ellipse
def visualize(**images):
    """Plot images in one row."""
    angles = list()
    
    n = len(images)
    plt.figure(figsize=(32, 16))
    for i, (name, image) in enumerate(images.items()):
        if i==0 and image.shape[0] == 3:
            image_main = image.permute(1,2,0).numpy()#.int()
        if torch.is_tensor(image) and image.shape[0] == 3:
            image = image.permute(1,2,0).numpy()#.int()
        else:
            try:
                image = image.numpy()
            except:
                pass
            
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        # plt.title()
        plt.imshow(image_main,cmap='gray')        
        plt.imshow(image,cmap='gray',alpha=0.4)
        if i == 1:
            head = head_direction(image)
        if i>=1:           
            major = True if i <=4 else False
            clockwise_plus = True if i==1 else False
#             x0,y0,angle = PCA_axis(image,major)
            x0,y0,angle = inertia_axis(image,major)
            plt.title(' '.join(name.split('_')).title()+'_MMI : {:.2f} degrees'.format(refine_degree(angle,head,clockwise_plus=clockwise_plus)))
            image = kornia.morphology.opening(torch.tensor(image).unsqueeze(0).unsqueeze(0),torch.ones(5,5))
            image = image.squeeze()
            draw_axis(image_main, image, x0, y0, angle)
    plt.tight_layout()
    plt.show()

def draw_axis(image, mask, x0, y0, radian_degree):
    """
    radian_degree
    """
    length = 320
    
    x1 = x0 + math.cos(radian_degree) * 0.5 * length
    y1 = y0 + math.sin(radian_degree) * 0.5 * length
    x2 = x0 - math.cos(radian_degree) * 0.5 * length
    y2 = y0 - math.sin(radian_degree) * 0.5 * length
    x3 = x0 + math.sin(radian_degree) * 0.5 * length
    y3 = y0 - math.cos(radian_degree) * 0.5 * length
    x4 = x0 - math.sin(radian_degree) * 0.5 * length
    y4 = y0 + math.cos(radian_degree) * 0.5 * length

    plt.imshow(image,cmap='gray')
    plt.imshow(mask,cmap='gray',alpha=0.4)
    plt.contour(mask,colors='#FD8A02',alpha=.5)
    angle = - np.tan((y0 - y2) / (x0 - x2))
    plt.plot((x0, x1), (y0, y1), '#01686D', linewidth=3)
    plt.plot((x0, x2), (y0, y2), '#01686D', linewidth=3)
    plt.plot(x0, y0, '.g', markersize=15)
    
import numpy as np
import matplotlib.pyplot as plt

from scipy import ndimage

def raw_moment(data, iord, jord):
    nrows, ncols = data.shape
    y, x = np.mgrid[:nrows, :ncols]
    data = data * x**iord * y**jord
    return data.sum() 

def inertia_axis(data, major):
    data_sum = data.sum()
    m10 = raw_moment(data, 1, 0)
    m01 = raw_moment(data, 0, 1)
    x_bar = m10 / data_sum
    y_bar = m01 / data_sum
    u11 = (raw_moment(data, 1, 1) - x_bar * m01) / data_sum
    u20 = (raw_moment(data, 2, 0) - x_bar * m10) / data_sum
    u02 = (raw_moment(data, 0, 2) - y_bar * m01) / data_sum
    
    if major == True:
        angle = 0.5 * np.arctan(2 * u11 / (u20 - u02))
    else:
        angle = 0.5 * np.arctan(2 * u11 / (u20 - u02)) # + np.pi/2
    
    return x_bar, y_bar, angle

def ellipse_axis(image, major=True):
    try:
        image = image.numpy()
    except:
        pass
    
    image = image.astype(np.uint8)
    regions = regionprops(image)
    for props in regions:
        y0, x0 = props.centroid
        orientation = props.orientation
        x1 = x0 + math.sin(orientation) * 0.5 * 300
        y1 = y0 + math.cos(orientation) * 0.5 * 300
        x2 = x0 - math.sin(orientation) * 0.5 * 300
        y2 = y0 - math.cos(orientation) * 0.5 * 300
        x3 = x0 + math.cos(orientation) * 0.5 * 150
        y3 = y0 - math.sin(orientation) * 0.5 * 150
        x4 = x0 - math.cos(orientation) * 0.5 * 150
        y4 = y0 + math.sin(orientation) * 0.5 * 150
        
        if major:
            angle = np.tan((y0 - y2) / (x0 - x2))
        else:
            angle = np.tan((y0 - y3) / (x0 - x3))

    return x0, y0, angle

import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.decomposition import PCA

def PCA_axis(mask_2d,major=True):
    
    mask_2d = np.rot90(mask_2d,2)
    mask_2d = np.flip(mask_2d)
    b= np.nonzero(mask_2d == 1)
    b = np.array(b)
    b = np.transpose(b) 
    b = np.flip(b) 
    X = b
    
#     # visualize
#     plt.axis('equal')
#     plt.scatter(X[:,0],X[:,1]);    

    pca=PCA(2)
    pca.fit(X)

#     print("Principal axes:", pca.components_)
#     print("Explained variance:", pca.explained_variance_)
#     print("Principal axes angle:",angle)
#     print("Mean:", pca.mean_)

    if major == True:
        angle = np.arctan(pca.components_[0][1]/pca.components_[0][0])
    else:
        angle = np.arctan(pca.components_[0][1]/pca.components_[0][0]) - np.pi/2
        
    x0,y0 = pca.mean_
    return x0, y0, angle

def refine_degree(radian, head, clockwise_plus=True):
    while radian <= -np.pi/2:
        radian += np.pi
    while radian >= np.pi/2:
        radian -= np.pi
    
    degree = np.degrees(radian)

    if head=='left' and clockwise_plus==True:
        return degree
    elif head=='right' and clockwise_plus==True:
        return -degree
    elif head=='left' and clockwise_plus==False:
        return -degree
    elif head=='right' and clockwise_plus==False:
        return degree
    
def head_direction(mask_1_2d):
    y,x = mask_1_2d.shape  
    x0,y0,_ = inertia_axis(mask_1_2d,True)
    ratio = x0/x
    if ratio > 0.5: 
        head = 'left'
    else:
        head = 'right'
    return head

In [ ]:
# idx = 4
# major = True if idx <4 else False
# head = head_direction((y_seg[0]))
# clockwise_plus = True if idx==0 else False

# # x0,y0,angle = PCA_axis(x,major)
# # plt.title('MMI : {:.2f} degrees'.format(refine_degree(angle,head,clockwise_plus=clockwise_plus)))
# # print(major,head)

# plt.figure(figsize=(20,20))
# plt.subplot(131)
# x0,y0,angle = PCA_axis(y_seg[idx],major)
# print(refine_degree(angle,head,clockwise_plus=clockwise_plus))
# # plt.title('PCA_{:.3f} degrees'.format(refine_degree(angle)))
# plt.axis('off')
# draw_axis(x[0], y_seg[idx].numpy(), x0, y0, angle)

# plt.subplot(132)
# x0,y0,angle = inertia_axis(y_seg[idx],True) 
# print(refine_degree(angle,head,clockwise_plus=clockwise_plus))
# plt.axis('off')
# draw_axis(x[0], y_seg[idx].numpy(), x0, y0, angle)

# plt.subplot(133)
# x0,y0,angle = ellipse_axis(y_seg[idx],major)
# print(refine_degree(angle,head,clockwise_plus=clockwise_plus))
# plt.axis('off')
# draw_axis(x[0], y_seg[idx].numpy(), x0, y0, angle)

# plt.tight_layout()

### Dataloader

Writing helper class for data extraction, tranformation and preprocessing  
https://pytorch.org/docs/stable/data

### Augmentations

In [ ]:
# # Augmentation
# import albumentations as albu
# def get_training_augmentation():
#     train_transform = [
        
#         albu.HorizontalFlip(p=0.5),
#         albu.ShiftScaleRotate(scale_limit=0.05, rotate_limit=10, shift_limit=0.02, border_mode=4, p=.5),
#         albu.IAAAdditiveGaussianNoise(scale=(1, 5),p=0.2),
#         albu.IAAPerspective(scale=(0.01, 0.02),p=0.5),
        
#         albu.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), brightness_by_max=False, p=0.5),
#         albu.RandomGamma(gamma_limit=(80,120), p=.5),
#         albu.RandomToneCurve(scale=0.1,p=.5), 
#         albu.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=.5),

#         albu.OneOf(
#             [
#                 albu.IAASharpen(p=.5),        
#                 albu.GaussNoise(var_limit=0.02, mean=0, p=0.5),
#                 albu.MultiplicativeNoise(multiplier=(0.98, 1.02), p=0.5),
#                 albu.ISONoise(color_shift=(0.01, 0.02),intensity=(0.1, 0.3),p=0.5),
#             ],
#             p=0.5,
#         ),
        
#         albu.PadIfNeeded(2500, 2500, border_mode=cv2.BORDER_CONSTANT, value=0, always_apply=True),
#         albu.CenterCrop(1024, 2048, always_apply=True),
# #         albu.PadIfNeeded(2560, 2560, border_mode=cv2.BORDER_CONSTANT, value=0, always_apply=True),
# #         albu.CenterCrop(1280, 2560, always_apply=True),
#         albu.Resize(512, 1024, interpolation=cv2.INTER_CUBIC, always_apply=True),
#         albu.CLAHE(p=1,always_apply=True),
#     ]
#     return albu.Compose(train_transform)


# def get_validation_augmentation():
#     """Add paddings to make image shape divisible by 32"""
#     test_transform = [
#         # albu.CLAHE(clip_limit=(2,2),p=1,always_apply=True),
#         # albu.PadIfNeeded(2500, 2500, border_mode=cv2.BORDER_CONSTANT, value=0, always_apply=True),
#         # albu.CenterCrop(1024, 2048, always_apply=True),
#         # albu.PadIfNeeded(2560, 2560, border_mode=cv2.BORDER_CONSTANT, value=0, always_apply=True),
# #         albu.CenterCrop(1280, 2560, always_apply=True),
#         # albu.Resize(512, 1024, interpolation=cv2.INTER_CUBIC, always_apply=True),
#     ]
#     return albu.Compose(test_transform)

In [ ]:
# Training & Validation Transform chain
import os
import torch
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from monai.losses import DiceCELoss
from monai.inferers import sliding_window_inference
from monai.transforms import *

from monai.config import print_config
from monai.metrics import DiceMetric

from monai.data import (
    Dataset,
    DataLoader,
    CacheDataset,
    decollate_batch,
)

def check(x):
    print(x.shape)
    return x

prob=0.3
patch_size= (512,1024)
train_transforms = Compose(
    [
        LoadImaged(keys=["image", "seg"]),
        # AddChanneld(keys=["image"]),
        Lambdad(keys='image', func=lambda x: np.transpose(x)),       
        Lambdad(keys='image', func=lambda x: cv2.cvtColor(x,cv2.COLOR_GRAY2RGB)),
        Lambdad(keys='image', func=lambda x: np.moveaxis(x,-1,0)),
        Lambdad(keys='seg', func=lambda x: np.moveaxis(x,-1,0)),
        
        # EnsureChannelFirstd(keys=["image", "seg"]),
        ScaleIntensityRanged(keys=["image"],a_min=0, a_max=255, b_min=0.0, b_max=1.0, clip=True,),
        
        # Spacingd(
        #     keys=["image", "seg"],
        #     pixdim=(1.5, 1.5, 2.0),
        #     mode=("bilinear", "nearest"),
        # ),
        
        RandAdjustContrastd(keys=["image"],gamma=(0.5, 4.5), prob=prob),
        RandShiftIntensityd(keys=["image"],offsets=(-0.3, 0.3), prob=prob),
        RandScaleIntensityd(keys=["image"],factors=(-0.3, 0.3), prob=prob),
        RandStdShiftIntensityd(keys=['image'],factors=(-5, 5), prob=prob),   
        RandHistogramShiftd(keys=["image"], num_control_points=20, prob=prob),
        # RandLambdad(keys='image', func=lambda x: 1-x, prob=prob),
        # ]),
        
        # OneOf(transforms=[
        RandGaussianSmoothd(keys=["image"],sigma_x=(0.25, 1.5), sigma_y=(0.25, 1.5), sigma_z=(0.25, 1.5), prob=prob),
        RandGaussianSharpend(keys=["image"],sigma1_x=(0.5, 1.0), sigma1_y=(0.5, 1.0), sigma1_z=(0.5, 1.0), sigma2_x=0.5, sigma2_y=0.5, sigma2_z=0.5, alpha=(10.0, 30.0), prob=prob),
        # ]),
            
        # OneOf(transforms=[
        RandGaussianNoised(keys=["image"],mean=0.0, std=0.1,prob=prob),
        RandBiasFieldd(keys=["image"], degree=3, coeff_range=(0.0, 0.1), prob=prob),
        RandGibbsNoised(keys=["image"], alpha=(0.0, 1.0), prob=prob),
        # RandKSpaceSpikeNoised(keys=["image"], global_prob=1.0, prob=prob),
        # ]),        
        
        RandZoomd(keys=["image","seg"], min_zoom=0.7, max_zoom=1.2, padding_mode=["constant","constant"], prob=prob),
        RandRotated(keys=["image","seg"], range_x=[-.2,.2], range_y=[-.2,.2], range_z=[-.2,.2], prob=prob),
        RandAffined(keys=["image","seg"], rotate_range=None, shear_range=[-.2,.2], translate_range=[-.15,.15], padding_mode=['zeros','zeros'], prob=1),
        Rand2DElasticd(keys=["image","seg"], spacing=(100,100), magnitude_range=(0.1,3), prob=prob),
        # CropForegroundd(keys=["image", "seg"], source_key="image"),
        
        RandFlipd(
            keys=["image", "seg"],
            spatial_axis=[0,1], # [1]
            prob=prob,
        ),
        
        Resized(keys=["image","seg"],spatial_size=1024, size_mode='longest',mode=['bicubic','nearest']),
        Lambdad(keys=['image'], func=lambda x: clahe(x,0)),
        RandLambdad(keys=['image'], func=lambda x: 1-x, prob=prob),
        SpatialPadd(keys=["image","seg"], method='symmetric', spatial_size=(512, 1024)),
        # ResizeWithPadOrCropd(keys=["image","seg"],spatial_size=[768, 1536],mode=['constant','constant']),
        # CropForegroundd(keys=["image","seg"],source_key="seg", margin = 350,),
        # Resized(keys=["image","seg"],spatial_size=1024, size_mode='longest',mode=['bicubic','nearest']),
        # RandCropByPosNegLabeld(
        #     keys=["image", "seg"],
        #     label_key="seg",
        #     spatial_size=(512, 1024),
        #     pos=1,
        #     neg=.5,
        #     num_samples=1,
        #     image_key="image",
        #     image_threshold=0,
        # ),
        # Resized(keys=["image","seg"],spatial_size=1024, size_mode='longest'),
        ToTensord(keys=["image", "seg"]),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["image", "seg"]),
        # EnsureChannelFirst
        # AddChanneld(keys=["image"]),
        # Lambdad(keys='image', func=lambda x: check(x)),
        Lambdad(keys='image', func=lambda x: np.transpose(x)),
        Lambdad(keys='image', func=lambda x: cv2.cvtColor(x,cv2.COLOR_GRAY2RGB)),
        Lambdad(keys='image', func=lambda x: np.moveaxis(x,-1,0)),
        Lambdad(keys='seg', func=lambda x: np.moveaxis(x,-1,0)),
        ScaleIntensityRanged(keys=["image"],a_min=0, a_max=255, b_min=0.0, b_max=1.0, clip=True,),
        Resized(keys=["image","seg"],spatial_size=1024, size_mode='longest',mode=['bicubic','nearest']),
        Lambdad(keys='image', func=lambda x: clahe(x,0)),
        # CropForegroundd(keys=["image","seg"], source_key="seg", margin = 300),
        CropForegroundd(keys=["image","seg"], source_key="seg", margin = 200),
        SpatialPadd(keys=["image","seg"], method='symmetric', spatial_size=(512, 1024)),
        CenterSpatialCropd(keys=["image","seg"], roi_size=(512, 1024)), # Others
        # SpatialCropd(keys=["image","seg"], roi_start=(0, 0), roi_end=(512, 1024)), # MMA
        ToTensord(keys=["image", "seg"]),
    ]
)

import mclahe

def clahe(img, dim = -1, adaptive_hist_range=False):
    """
    input 1 numpy shape image (H x W x (D) x C)
    """
    temp = np.zeros_like(img)
    if dim == -1:
        for idx in range(temp.shape[-1]):
            temp[...,idx] = mclahe.mclahe(img[...,idx], kernel_size=None, n_bins=128, clip_limit=0.04, adaptive_hist_range=adaptive_hist_range)
    elif dim == 0:
        for idx in range(temp.shape[0]):
            temp[idx] = mclahe.mclahe(img[idx], kernel_size=None, n_bins=128, clip_limit=0.04, adaptive_hist_range=adaptive_hist_range)
    
    return temp

import sklearn
import sklearn.model_selection
# DATA_DIR = 'ActiveLearning/1st'
# DATA_DIR = 'ActiveLearning/2nd'
# DATA_DIR = 'ActiveLearning/3rd'
DATA_DIR = 'ActiveLearning/4th'

# DATA_DIR = 'NonActiveLearning/1st'
# DATA_DIR = 'NonActiveLearning/2nd'
# DATA_DIR = 'NonActiveLearning/3rd'
# DATA_DIR = 'NonActiveLearning/4th'

x_train = natsort.natsorted(glob.glob(os.path.join(DATA_DIR, 'x_train','*.png')))
y_train = natsort.natsorted(glob.glob(os.path.join(DATA_DIR, 'y_train','*.npy')))

# sklearn의 train_test_split를 사용해 데이터를 나눕니다. 반복성 있게 데이터가 나누어 집니다.
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train,test_size=0.1,random_state=42)

x_test = natsort.natsorted(glob.glob(os.path.join(DATA_DIR, 'x_test','*.png')))
y_test = natsort.natsorted(glob.glob(os.path.join(DATA_DIR, 'y_test','*.npy')))
train_files = [{"image": img, "seg": seg, "fname": img} for img, seg in zip(x_train, y_train)]
val_files = [{"image": img, "seg": seg, "fname": img} for img, seg in zip(x_valid, y_valid)]
test_files = [{"image": img, "seg": seg, "fname": img} for img, seg in zip(x_test, y_test)]

DATA_DIR = 'ExternalObserver/GP1/'
x_testGP1 = natsort.natsorted(glob.glob(os.path.join(DATA_DIR, 'x_test','*.png')))
y_testGP1 = natsort.natsorted(glob.glob(os.path.join(DATA_DIR, 'y_test','*.npy')))

DATA_DIR = 'ExternalObserver/GP2/'
x_testGP2 = natsort.natsorted(glob.glob(os.path.join(DATA_DIR, 'x_test','*.png')))
y_testGP2 = natsort.natsorted(glob.glob(os.path.join(DATA_DIR, 'y_test','*.npy')))

DATA_DIR = 'ExternalObserver/OS/'
x_testOS = natsort.natsorted(glob.glob(os.path.join(DATA_DIR, 'x_test','*.png')))
y_testOS = natsort.natsorted(glob.glob(os.path.join(DATA_DIR, 'y_test','*.npy')))

x_etest = natsort.natsorted(glob.glob(os.path.join('ExternalValidation', 'LERA_WBLR_seg','*.png')))
y_etest = natsort.natsorted(glob.glob(os.path.join('ExternalValidation', 'LERA_WBLR_seg','*.npy')))

x_etest2 = natsort.natsorted(glob.glob(os.path.join('ExternalValidation', 'AMC_100_WBLR_seg','refined','*.png')))
y_etest2 = natsort.natsorted(glob.glob(os.path.join('ExternalValidation', 'AMC_100_WBLR_seg','refined','*.npy')))

train_files = [{"image": img, "seg": seg, "fname": img} for img, seg in zip(x_train, y_train)]
val_files = [{"image": img, "seg": seg, "fname": img} for img, seg in zip(x_valid, y_valid)]
test_files = [{"image": img, "seg": seg, "fname": img} for img, seg in zip(x_test, y_test)]
testGP1_files = [{"image": img, "seg": seg, "fname": img} for img, seg in zip(x_testGP1, y_testGP1)]
testGP2_files = [{"image": img, "seg": seg, "fname": img} for img, seg in zip(x_testGP2, y_testGP2)]
testOS_files = [{"image": img, "seg": seg, "fname": img} for img, seg in zip(x_testOS, y_testOS)]
etest_files = [{"image": img, "seg": seg, "fname": img} for img, seg in zip(x_etest, y_etest)]
etest2_files = [{"image": img, "seg": seg, "fname": img} for img, seg in zip(x_etest2, y_etest2)]

In [ ]:
# train_ds = CacheDataset(
#     data=train_files,
#     transform=train_transforms,
#     cache_num=8,
#     cache_rate=1.0,
#     num_workers=4,
# )
# train_loader = DataLoader(
#     train_ds, batch_size=1, shuffle=True, num_workers=4, pin_memory=True
# )

# val_ds = CacheDataset(
#     data=val_files,
#     transform=val_transforms,
#     cache_num=4,
#     cache_rate=1.0,
#     num_workers=4
# )
# val_loader = DataLoader(
#     val_ds, batch_size=1, shuffle=False, num_workers=2, pin_memory=True
# )

test_ds = Dataset(
    data=test_files,
    transform=val_transforms,
)

test_loader = DataLoader(
    test_ds, batch_size=1, shuffle=False, num_workers=1, pin_memory=True
)

testGP1_ds = Dataset(
    data=testGP1_files,
    transform=val_transforms,
)

testGP1_loader = DataLoader(
    testGP1_ds, batch_size=1, shuffle=False, num_workers=1, pin_memory=True
)

testGP2_ds = Dataset(
    data=testGP2_files,
    transform=val_transforms,
)

testGP2_loader = DataLoader(
    testGP2_ds, batch_size=1, shuffle=False, num_workers=1, pin_memory=True
)

testOS_ds = Dataset(
    data=testOS_files,
    transform=val_transforms,
)

testOS_loader = DataLoader(
    testOS_ds, batch_size=1, shuffle=False, num_workers=1, pin_memory=True
)

etest_ds = Dataset(
    data=etest_files,
    transform=val_transforms,
)

etest_loader = DataLoader(
    etest_ds, batch_size=1, shuffle=False, num_workers=1, pin_memory=True
)

etest2_ds = Dataset(
    data=etest2_files,
    transform=val_transforms,
)

etest2_loader = DataLoader(
    etest2_ds, batch_size=1, shuffle=False, num_workers=1, pin_memory=True
)

In [ ]:
# iterator = iter(train_loader)
# iterator = iter(val_loader)
# iterator = iter(test_loader)
# iterator = iter(etest_loader)
batch = iterator.next()
x = batch['image']
y = batch['seg']
print(x.shape, y.shape)
for idx in range(len(x)):
    plt.imshow(x[idx,0],cmap='gray')
    plt.show()

In [ ]:
iterator = iter(test_loader)
batch = iterator.next()
x = batch['image']
y = batch['seg']
print(x.shape, y.shape)
for idx in range(len(x)):
    plt.imshow(x[idx,0],cmap='gray')
    plt.show()

In [ ]:
iterator = iter(testGP1_loader)
batch = iterator.next()
x = batch['image']
y = batch['seg']
print(x.shape, y.shape)
for idx in range(len(x)):
    plt.imshow(x[idx,0],cmap='gray')
    plt.show()

In [ ]:
iterator = iter(testGP2_loader)
batch = iterator.next()
x = batch['image']
y = batch['seg']
print(x.shape, y.shape)
for idx in range(len(x)):
    plt.imshow(x[idx,0],cmap='gray')
    plt.show()

In [ ]:
iterator = iter(testOS_loader)
batch = iterator.next()
x = batch['image']
y = batch['seg']
print(x.shape, y.shape)
for idx in range(len(x)):
    plt.imshow(x[idx,0],cmap='gray')
    plt.show()

In [ ]:
# iterator = iter(train_loader)
# iterator = iter(val_loader)
# iterator = iter(test_loader)
iterator = iter(etest_loader)
batch = iterator.next()
x = batch['image']
y = batch['seg']
print(x.shape, y.shape)
for idx in range(len(x)):
    plt.imshow(x[idx,0],cmap='gray')
    plt.show()

In [ ]:
iterator = iter(etest2_loader)
batch = iterator.next()
x = batch['image']
y = batch['seg']
print(x.shape, y.shape)
for idx in range(len(x)):
    plt.imshow(x[idx,0],cmap='gray')
    plt.show()

In [ ]:
# import sklearn
# import sklearn.model_selection 
# from sklearn.model_selection import train_test_split

# x_train = natsort.natsorted(glob.glob(os.path.join(STAGE, 'x_train','*.png')))
# y_train = natsort.natsorted(glob.glob(os.path.join(STAGE, 'y_train','*.npy')))

# # x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train,test_size=0.1, random_state=42)

# x_test = natsort.natsorted(glob.glob(os.path.join(STAGE, 'x_test','*.png')))
# y_test = natsort.natsorted(glob.glob(os.path.join(STAGE, 'y_test','*.npy')))

# x_inference = natsort.natsorted(glob.glob(os.path.join(STAGE, 'x_inference','*')))
# y_inference = natsort.natsorted(glob.glob(os.path.join(STAGE, 'y_inference','*')))

# x_etest = natsort.natsorted(glob.glob(os.path.join('ExternalValidation','LERA_WBLR_seg','*.png')))
# y_etest = natsort.natsorted(glob.glob(os.path.join('ExternalValidation','LERA_WBLR_seg','*.npy')))

# train_dataset = Dataset(
#     x_train, 
#     y_train, 
#     augmentation=get_validation_augmentation(), 
# )


# valid_dataset = Dataset(
#     x_valid, 
#     y_valid, 
#     augmentation=get_validation_augmentation(), 
# ) 

# test_dataset = Dataset(
#     x_test, 
#     y_test, 
#     augmentation=get_validation_augmentation(), 
# )

# etest_dataset = Dataset(
#     x_etest, 
#     y_etest, 
#     augmentation=get_validation_augmentation(), 
# )

# inference_dataset = Dataset(
#     x_inference, 
#     y_inference, 
#     augmentation=get_validation_augmentation(), 
# )

# train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=1)
# valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=1)
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=1)
# etest_loader = DataLoader(etest_dataset, batch_size=1, shuffle=False, num_workers=1)
# inference_loader = DataLoader(inference_dataset, batch_size=1, shuffle=False, num_workers=1)

In [ ]:
# trainset
batch = next(iter(train_loader))
x = batch['image'][0]
y_seg = batch['seg'][0]
fname = batch['fname'][0]
print(fname,x.shape,y_seg.shape)
visualize(image=x, mask1=y_seg[0], mask2=y_seg[1], mask3=y_seg[2], mask4=y_seg[3], mask5=y_seg[4])

In [ ]:
# testset
batch = next(iter(test_loader))
x = batch['image'][0]
y_seg = batch['seg'][0]
fname = batch['fname'][0]
print(fname)

visualize(image=x, mask1=y_seg[0], mask2=y_seg[1], mask3=y_seg[2], mask4=y_seg[3], mask5=y_seg[4])

In [ ]:
# testset
batch = next(iter(testGP1_loader))
x = batch['image'][0]
y_seg = batch['seg'][0]
fname = batch['fname'][0]
print(fname)

visualize(image=x, mask1=y_seg[0], mask2=y_seg[1], mask3=y_seg[2], mask4=y_seg[3], mask5=y_seg[4])

In [ ]:
# testset

batch = next(iter(etest_loader))
x = batch['image'][0]
y_seg = batch['seg'][0]
fname = batch['fname'][0]
print(fname, x.shape)

visualize(image=x, mask1=y_seg[0], mask2=y_seg[1], mask3=y_seg[2], mask4=y_seg[3], mask5=y_seg[4])

In [ ]:
# # testset

# batch = next(iter(inference_loader))
# x = batch['x'][0]
# y_seg = batch['y_seg'][0]
# fname = batch['fname'][0]
# print(fname)

# visualize(image=x, mask1=y_seg[0], mask2=y_seg[1], mask3=y_seg[2], mask4=y_seg[3], mask5=y_seg[4])

In [ ]:
# show all
angles_total = list()
for idx, batch in enumerate(train_loader):
    x = batch['x'][0]
    y_seg = batch['y_seg'][0]
    fname = batch['fname'][0]
    print(idx, fname)
    angles = visualize(image=x, mask1=y_seg[0], mask2=y_seg[1], mask3=y_seg[2], mask4=y_seg[3], mask5=y_seg[4])

In [ ]:
# show all
angles_total = list()
for idx, batch in enumerate(test_loader):
    x = batch['image'][0]
    y_seg = batch['seg'][0]
    fname = batch['fname'][0]
    print(idx, fname)
    visualize(image=x, mask1=y_seg[0], mask2=y_seg[1], mask3=y_seg[2], mask4=y_seg[3], mask5=y_seg[4])

In [ ]:
# show all
angles_total = list()
for idx, batch in enumerate(etest_loader):
    x = batch['image'][0]
    y_seg = batch['seg'][0]
    fname = batch['fname'][0]
    print(idx, fname)
    visualize(image=x, mask1=y_seg[0], mask2=y_seg[1], mask3=y_seg[2], mask4=y_seg[3], mask5=y_seg[4])

In [ ]:
angles_1 = list()
angles_2 = list()
angles_3 = list()
angles_4 = list()
angles_5 = list()

for idx in range(len(angles_total)):
    angles_1.append(angles_total[idx][0])
    angles_2.append(angles_total[idx][1])
    angles_3.append(angles_total[idx][2])
    angles_4.append(angles_total[idx][3])
    angles_5.append(angles_total[idx][4])
df = pd.DataFrame([files,angles_1,angles_2,angles_3,angles_4,angles_5],['fname','angles_1','angles_2','angles_3','angles_4','angles_5'])
df = df.T
# df.to_csv('angles_train_eclipse.csv',index=False)
df.to_csv('angles_train_inertia_axis.csv',index=False)

In [ ]:
# show all
for idx, batch in enumerate(valid_loader):
    x = batch['x'][0]
    y_seg = batch['y_seg'][0]
    fname = batch['fname'][0]
    print(fname)

    visualize(image=x, mask1=y_seg[0], mask2=y_seg[1], mask3=y_seg[2], mask4=y_seg[3], mask5=y_seg[4])

In [ ]:
# show all
for idx, batch in enumerate(test_loader):
    x = batch['x'][0]
    y_seg = batch['y_seg'][0]
    fname = batch['fname'][0]
    print(fname)

    visualize(image=x, mask1=y_seg[0], mask2=y_seg[1], mask3=y_seg[2], mask4=y_seg[3], mask5=y_seg[4])

In [ ]:
plt.imshow(1-y_seg[0],cmap='gray')
plt.xticks([])
plt.yticks([])

In [ ]:
x = batch['x'][0]
y_seg = batch['y_seg'][0]
fname = batch['fname'][0]
#     print(fname)
plt.imshow(x[0],cmap='gray')
# y = y_seg[0]+y_seg[1]*2+y_seg[2]*3+y_seg[3]*4+y_seg[4]*5
y = y_seg[0]#+y_seg[1]+y_seg[2]+y_seg[3]+y_seg[4]
plt.imshow(y,cmap='gray',alpha=0.5)
plt.xticks([])
plt.yticks([])

In [ ]:
# def ellipse_angle(image, major=True):
#     image = image.astype(np.uint8)
#     regions = regionprops(image)
#     for props in regions:
#         y0, x0 = props.centroid
#         orientation = props.orientation
#         x1 = x0 + math.sin(orientation) * 0.5 * props.major_axis_length
#         y1 = y0 + math.cos(orientation) * 0.5 * props.major_axis_length
#         x2 = x0 - math.sin(orientation) * 0.5 * props.major_axis_length
#         y2 = y0 - math.cos(orientation) * 0.5 * props.major_axis_length
#         x3 = x0 + math.cos(orientation) * 0.5 * props.minor_axis_length
#         y3 = y0 - math.sin(orientation) * 0.5 * props.minor_axis_length
#         x4 = x0 - math.cos(orientation) * 0.5 * props.minor_axis_length
#         y4 = y0 + math.sin(orientation) * 0.5 * props.minor_axis_length
#         if major:
#             angle = - np.tan((y0 - y2) / (x0 - x2))
#             plt.plot((x0, x1), (y0, y1), 'black', linewidth=1)
#             plt.plot((x0, x2), (y0, y2), 'black', linewidth=1)
#             plt.plot(x0, y0, '.g', markersize=15)
#         else:
#             angle = - np.tan((y0 - y3) / (x0 - x3))
#             plt.plot((x0, x3), (y0, y3), 'black', linewidth=1)
#             plt.plot((x0, x4), (y0, y4), 'black', linewidth=1)
#             plt.plot(x0, y0, '.r', markersize=15)

#         if angle >= np.pi:
#             angle = angle - np.pi
#         elif angle <= - np.pi:
#             angle = np.pi + angle 
#         angle = np.degrees(angle)
#     return angle

In [ ]:
# plt.imshow(x[0],cmap='gray')
plt.figure(figsize=(16,16))
plt.imshow(y_seg[0],cmap='gray')
plt.xticks([])
plt.yticks([])

In [ ]:
plt.imshow(x[0],cmap='gray')
plt.imshow(y_seg[0],cmap='gray',alpha=0.5)

plt.xticks([])
plt.yticks([])

In [ ]:
for idx in range(5):
    major = True if idx<=3 else False
    print(ellipse_axis(y_seg[idx].numpy(),major=major))

In [ ]:
plt.imshow(y,cmap='gray')
plt.xticks([])
plt.yticks([])

## Create model and train

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import segmentation_models_pytorch as smp
# import tqdm
from tqdm import trange

In [ ]:
# ENCODER = 'resnet50'

# ENCODER_WEIGHTS = 'imagenet'
# ENCODER_WEIGHTS = None
# ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
# DEVICE = 'cuda'

# # create segmentation model with pretrained encoder

# model = smp.Unet(
#     encoder_name=ENCODER, 
#     encoder_weights=ENCODER_WEIGHTS,
#     in_channels=3,
#     classes=5, 
#     activation=ACTIVATION,
#     decoder_attention_type='scse',
# )

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class REBNCONV(nn.Module):
    def __init__(self,in_ch=3,out_ch=3,dirate=1):
        super(REBNCONV,self).__init__()

        self.conv_s1 = nn.Conv2d(in_ch,out_ch,3,padding=1*dirate,dilation=1*dirate)
        self.bn_s1 = nn.BatchNorm2d(out_ch)
        self.relu_s1 = nn.ReLU(inplace=True)

    def forward(self,x):

        hx = x
        xout = self.relu_s1(self.bn_s1(self.conv_s1(hx)))

        return xout

## upsample tensor 'src' to have the same spatial size with tensor 'tar'
def _upsample_like(src,tar):

    src = F.upsample(src,size=tar.shape[2:],mode='bilinear')

    return src


### RSU-7 ###
class RSU7(nn.Module):#UNet07DRES(nn.Module):

    def __init__(self, in_ch=3, mid_ch=12, out_ch=3):
        super(RSU7,self).__init__()

        self.rebnconvin = REBNCONV(in_ch,out_ch,dirate=1)

        self.rebnconv1 = REBNCONV(out_ch,mid_ch,dirate=1)
        self.pool1 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv2 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool2 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv3 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool3 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv4 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool4 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv5 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool5 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv6 = REBNCONV(mid_ch,mid_ch,dirate=1)

        self.rebnconv7 = REBNCONV(mid_ch,mid_ch,dirate=2)

        self.rebnconv6d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv5d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv4d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv3d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv2d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv1d = REBNCONV(mid_ch*2,out_ch,dirate=1)

    def forward(self,x):

        hx = x
        hxin = self.rebnconvin(hx)

        hx1 = self.rebnconv1(hxin)
        hx = self.pool1(hx1)

        hx2 = self.rebnconv2(hx)
        hx = self.pool2(hx2)

        hx3 = self.rebnconv3(hx)
        hx = self.pool3(hx3)

        hx4 = self.rebnconv4(hx)
        hx = self.pool4(hx4)

        hx5 = self.rebnconv5(hx)
        hx = self.pool5(hx5)

        hx6 = self.rebnconv6(hx)

        hx7 = self.rebnconv7(hx6)

        hx6d =  self.rebnconv6d(torch.cat((hx7,hx6),1))
        hx6dup = _upsample_like(hx6d,hx5)

        hx5d =  self.rebnconv5d(torch.cat((hx6dup,hx5),1))
        hx5dup = _upsample_like(hx5d,hx4)

        hx4d = self.rebnconv4d(torch.cat((hx5dup,hx4),1))
        hx4dup = _upsample_like(hx4d,hx3)

        hx3d = self.rebnconv3d(torch.cat((hx4dup,hx3),1))
        hx3dup = _upsample_like(hx3d,hx2)

        hx2d = self.rebnconv2d(torch.cat((hx3dup,hx2),1))
        hx2dup = _upsample_like(hx2d,hx1)

        hx1d = self.rebnconv1d(torch.cat((hx2dup,hx1),1))

        return hx1d + hxin

### RSU-6 ###
class RSU6(nn.Module):#UNet06DRES(nn.Module):

    def __init__(self, in_ch=3, mid_ch=12, out_ch=3):
        super(RSU6,self).__init__()

        self.rebnconvin = REBNCONV(in_ch,out_ch,dirate=1)

        self.rebnconv1 = REBNCONV(out_ch,mid_ch,dirate=1)
        self.pool1 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv2 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool2 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv3 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool3 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv4 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool4 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv5 = REBNCONV(mid_ch,mid_ch,dirate=1)

        self.rebnconv6 = REBNCONV(mid_ch,mid_ch,dirate=2)

        self.rebnconv5d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv4d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv3d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv2d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv1d = REBNCONV(mid_ch*2,out_ch,dirate=1)

    def forward(self,x):

        hx = x

        hxin = self.rebnconvin(hx)

        hx1 = self.rebnconv1(hxin)
        hx = self.pool1(hx1)

        hx2 = self.rebnconv2(hx)
        hx = self.pool2(hx2)

        hx3 = self.rebnconv3(hx)
        hx = self.pool3(hx3)

        hx4 = self.rebnconv4(hx)
        hx = self.pool4(hx4)

        hx5 = self.rebnconv5(hx)

        hx6 = self.rebnconv6(hx5)


        hx5d =  self.rebnconv5d(torch.cat((hx6,hx5),1))
        hx5dup = _upsample_like(hx5d,hx4)

        hx4d = self.rebnconv4d(torch.cat((hx5dup,hx4),1))
        hx4dup = _upsample_like(hx4d,hx3)

        hx3d = self.rebnconv3d(torch.cat((hx4dup,hx3),1))
        hx3dup = _upsample_like(hx3d,hx2)

        hx2d = self.rebnconv2d(torch.cat((hx3dup,hx2),1))
        hx2dup = _upsample_like(hx2d,hx1)

        hx1d = self.rebnconv1d(torch.cat((hx2dup,hx1),1))

        return hx1d + hxin

### RSU-5 ###
class RSU5(nn.Module):#UNet05DRES(nn.Module):

    def __init__(self, in_ch=3, mid_ch=12, out_ch=3):
        super(RSU5,self).__init__()

        self.rebnconvin = REBNCONV(in_ch,out_ch,dirate=1)

        self.rebnconv1 = REBNCONV(out_ch,mid_ch,dirate=1)
        self.pool1 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv2 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool2 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv3 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool3 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv4 = REBNCONV(mid_ch,mid_ch,dirate=1)

        self.rebnconv5 = REBNCONV(mid_ch,mid_ch,dirate=2)

        self.rebnconv4d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv3d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv2d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv1d = REBNCONV(mid_ch*2,out_ch,dirate=1)

    def forward(self,x):

        hx = x

        hxin = self.rebnconvin(hx)

        hx1 = self.rebnconv1(hxin)
        hx = self.pool1(hx1)

        hx2 = self.rebnconv2(hx)
        hx = self.pool2(hx2)

        hx3 = self.rebnconv3(hx)
        hx = self.pool3(hx3)

        hx4 = self.rebnconv4(hx)

        hx5 = self.rebnconv5(hx4)

        hx4d = self.rebnconv4d(torch.cat((hx5,hx4),1))
        hx4dup = _upsample_like(hx4d,hx3)

        hx3d = self.rebnconv3d(torch.cat((hx4dup,hx3),1))
        hx3dup = _upsample_like(hx3d,hx2)

        hx2d = self.rebnconv2d(torch.cat((hx3dup,hx2),1))
        hx2dup = _upsample_like(hx2d,hx1)

        hx1d = self.rebnconv1d(torch.cat((hx2dup,hx1),1))

        return hx1d + hxin

### RSU-4 ###
class RSU4(nn.Module):#UNet04DRES(nn.Module):

    def __init__(self, in_ch=3, mid_ch=12, out_ch=3):
        super(RSU4,self).__init__()

        self.rebnconvin = REBNCONV(in_ch,out_ch,dirate=1)

        self.rebnconv1 = REBNCONV(out_ch,mid_ch,dirate=1)
        self.pool1 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv2 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool2 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv3 = REBNCONV(mid_ch,mid_ch,dirate=1)

        self.rebnconv4 = REBNCONV(mid_ch,mid_ch,dirate=2)

        self.rebnconv3d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv2d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv1d = REBNCONV(mid_ch*2,out_ch,dirate=1)

    def forward(self,x):

        hx = x

        hxin = self.rebnconvin(hx)

        hx1 = self.rebnconv1(hxin)
        hx = self.pool1(hx1)

        hx2 = self.rebnconv2(hx)
        hx = self.pool2(hx2)

        hx3 = self.rebnconv3(hx)

        hx4 = self.rebnconv4(hx3)

        hx3d = self.rebnconv3d(torch.cat((hx4,hx3),1))
        hx3dup = _upsample_like(hx3d,hx2)

        hx2d = self.rebnconv2d(torch.cat((hx3dup,hx2),1))
        hx2dup = _upsample_like(hx2d,hx1)

        hx1d = self.rebnconv1d(torch.cat((hx2dup,hx1),1))

        return hx1d + hxin

### RSU-4F ###
class RSU4F(nn.Module):#UNet04FRES(nn.Module):

    def __init__(self, in_ch=3, mid_ch=12, out_ch=3):
        super(RSU4F,self).__init__()

        self.rebnconvin = REBNCONV(in_ch,out_ch,dirate=1)

        self.rebnconv1 = REBNCONV(out_ch,mid_ch,dirate=1)
        self.rebnconv2 = REBNCONV(mid_ch,mid_ch,dirate=2)
        self.rebnconv3 = REBNCONV(mid_ch,mid_ch,dirate=4)

        self.rebnconv4 = REBNCONV(mid_ch,mid_ch,dirate=8)

        self.rebnconv3d = REBNCONV(mid_ch*2,mid_ch,dirate=4)
        self.rebnconv2d = REBNCONV(mid_ch*2,mid_ch,dirate=2)
        self.rebnconv1d = REBNCONV(mid_ch*2,out_ch,dirate=1)

    def forward(self,x):

        hx = x

        hxin = self.rebnconvin(hx)

        hx1 = self.rebnconv1(hxin)
        hx2 = self.rebnconv2(hx1)
        hx3 = self.rebnconv3(hx2)

        hx4 = self.rebnconv4(hx3)

        hx3d = self.rebnconv3d(torch.cat((hx4,hx3),1))
        hx2d = self.rebnconv2d(torch.cat((hx3d,hx2),1))
        hx1d = self.rebnconv1d(torch.cat((hx2d,hx1),1))

        return hx1d + hxin


##### U^2-Net ####
class U2NET(nn.Module):

    def __init__(self,in_ch=1,out_ch=5):
        super(U2NET,self).__init__()

        self.stage1 = RSU7(in_ch,32,64)
        self.pool12 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage2 = RSU6(64,32,128)
        self.pool23 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage3 = RSU5(128,64,256)
        self.pool34 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage4 = RSU4(256,128,512)
        self.pool45 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage5 = RSU4F(512,256,512)
        self.pool56 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage6 = RSU4F(512,256,512)

        # decoder
        self.stage5d = RSU4F(1024,256,512)
        self.stage4d = RSU4(1024,128,256)
        self.stage3d = RSU5(512,64,128)
        self.stage2d = RSU6(256,32,64)
        self.stage1d = RSU7(128,16,64)

        self.side1 = nn.Conv2d(64,out_ch,3,padding=1)
        self.side2 = nn.Conv2d(64,out_ch,3,padding=1)
        self.side3 = nn.Conv2d(128,out_ch,3,padding=1)
        self.side4 = nn.Conv2d(256,out_ch,3,padding=1)
        self.side5 = nn.Conv2d(512,out_ch,3,padding=1)
        self.side6 = nn.Conv2d(512,out_ch,3,padding=1)

        self.outconv = nn.Conv2d(6*out_ch,out_ch,1)

    def forward(self,x):

        hx = x

        #stage 1
        hx1 = self.stage1(hx)
        hx = self.pool12(hx1)

        #stage 2
        hx2 = self.stage2(hx)
        hx = self.pool23(hx2)

        #stage 3
        hx3 = self.stage3(hx)
        hx = self.pool34(hx3)

        #stage 4
        hx4 = self.stage4(hx)
        hx = self.pool45(hx4)

        #stage 5
        hx5 = self.stage5(hx)
        hx = self.pool56(hx5)

        #stage 6
        hx6 = self.stage6(hx)
        hx6up = _upsample_like(hx6,hx5)

        #-------------------- decoder --------------------
        hx5d = self.stage5d(torch.cat((hx6up,hx5),1))
        hx5dup = _upsample_like(hx5d,hx4)

        hx4d = self.stage4d(torch.cat((hx5dup,hx4),1))
        hx4dup = _upsample_like(hx4d,hx3)

        hx3d = self.stage3d(torch.cat((hx4dup,hx3),1))
        hx3dup = _upsample_like(hx3d,hx2)

        hx2d = self.stage2d(torch.cat((hx3dup,hx2),1))
        hx2dup = _upsample_like(hx2d,hx1)

        hx1d = self.stage1d(torch.cat((hx2dup,hx1),1))


        #side output
        d1 = self.side1(hx1d)

        d2 = self.side2(hx2d)
        d2 = _upsample_like(d2,d1)

        d3 = self.side3(hx3d)
        d3 = _upsample_like(d3,d1)

        d4 = self.side4(hx4d)
        d4 = _upsample_like(d4,d1)

        d5 = self.side5(hx5d)
        d5 = _upsample_like(d5,d1)

        d6 = self.side6(hx6)
        d6 = _upsample_like(d6,d1)

        d0 = self.outconv(torch.cat((d1,d2,d3,d4,d5,d6),1))

        return F.sigmoid(d0), F.sigmoid(d1), F.sigmoid(d2), F.sigmoid(d3), F.sigmoid(d4), F.sigmoid(d5), F.sigmoid(d6)

### U^2-Net small ###
class U2NETP(nn.Module):

    def __init__(self,in_ch=3,out_ch=1):
        super(U2NETP,self).__init__()

        self.stage1 = RSU7(in_ch,16,64)
        self.pool12 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage2 = RSU6(64,16,64)
        self.pool23 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage3 = RSU5(64,16,64)
        self.pool34 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage4 = RSU4(64,16,64)
        self.pool45 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage5 = RSU4F(64,16,64)
        self.pool56 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage6 = RSU4F(64,16,64)

        # decoder
        self.stage5d = RSU4F(128,16,64)
        self.stage4d = RSU4(128,16,64)
        self.stage3d = RSU5(128,16,64)
        self.stage2d = RSU6(128,16,64)
        self.stage1d = RSU7(128,16,64)

        self.side1 = nn.Conv2d(64,out_ch,3,padding=1)
        self.side2 = nn.Conv2d(64,out_ch,3,padding=1)
        self.side3 = nn.Conv2d(64,out_ch,3,padding=1)
        self.side4 = nn.Conv2d(64,out_ch,3,padding=1)
        self.side5 = nn.Conv2d(64,out_ch,3,padding=1)
        self.side6 = nn.Conv2d(64,out_ch,3,padding=1)

        self.outconv = nn.Conv2d(6*out_ch,out_ch,1)

    def forward(self,x):

        hx = x

        #stage 1
        hx1 = self.stage1(hx)
        hx = self.pool12(hx1)

        #stage 2
        hx2 = self.stage2(hx)
        hx = self.pool23(hx2)

        #stage 3
        hx3 = self.stage3(hx)
        hx = self.pool34(hx3)

        #stage 4
        hx4 = self.stage4(hx)
        hx = self.pool45(hx4)

        #stage 5
        hx5 = self.stage5(hx)
        hx = self.pool56(hx5)

        #stage 6
        hx6 = self.stage6(hx)
        hx6up = _upsample_like(hx6,hx5)

        #decoder
        hx5d = self.stage5d(torch.cat((hx6up,hx5),1))
        hx5dup = _upsample_like(hx5d,hx4)

        hx4d = self.stage4d(torch.cat((hx5dup,hx4),1))
        hx4dup = _upsample_like(hx4d,hx3)

        hx3d = self.stage3d(torch.cat((hx4dup,hx3),1))
        hx3dup = _upsample_like(hx3d,hx2)

        hx2d = self.stage2d(torch.cat((hx3dup,hx2),1))
        hx2dup = _upsample_like(hx2d,hx1)

        hx1d = self.stage1d(torch.cat((hx2dup,hx1),1))


        #side output
        d1 = self.side1(hx1d)

        d2 = self.side2(hx2d)
        d2 = _upsample_like(d2,d1)

        d3 = self.side3(hx3d)
        d3 = _upsample_like(d3,d1)

        d4 = self.side4(hx4d)
        d4 = _upsample_like(d4,d1)

        d5 = self.side5(hx5d)
        d5 = _upsample_like(d5,d1)

        d6 = self.side6(hx6)
        d6 = _upsample_like(d6,d1)

        d0 = self.outconv(torch.cat((d1,d2,d3,d4,d5,d6),1))

        return F.sigmoid(d0), F.sigmoid(d1), F.sigmoid(d2), F.sigmoid(d3), F.sigmoid(d4), F.sigmoid(d5), F.sigmoid(d6)
    
bce_loss = nn.BCELoss(size_average=True)

def muti_bce_loss_fusion(yhat, labels_v):
    d0, d1, d2, d3, d4, d5, d6 = yhat
    loss0 = bce_loss(d0,labels_v)
    loss1 = bce_loss(d1,labels_v)
    loss2 = bce_loss(d2,labels_v)
    loss3 = bce_loss(d3,labels_v)
    loss4 = bce_loss(d4,labels_v)
    loss5 = bce_loss(d5,labels_v)
    loss6 = bce_loss(d6,labels_v)

    loss = loss0 + loss1 + loss2 + loss3 + loss4 + loss5 + loss6
    # print("l0: %3f, l1: %3f, l2: %3f, l3: %3f, l4: %3f, l5: %3f, l6: %3f\n"%(loss0.data.item(),loss1.data.item(),loss2.data.item(),loss3.data.item(),loss4.data.item(),loss5.data.item(),loss6.data.item()))

    return loss

def bn2group(module):
    num_groups = 16 # hyper_parameter of GroupNorm
    # num_groups = 8 # hyper_parameter of GroupNorm
    module_output = module
    if isinstance(module, torch.nn.modules.batchnorm._BatchNorm):
        if module.num_features/num_groups <1:
            module_output = torch.nn.GroupNorm(1,
                                           module.num_features,
                                           module.eps, 
                                           module.affine,
                                          )
        else:
            module_output = torch.nn.GroupNorm(num_groups,
                               module.num_features,
                               module.eps, 
                               module.affine,
                                          )

        if module.affine:
            with torch.no_grad():
                module_output.weight = module.weight
                module_output.bias = module.bias
        module_output.running_mean = module.running_mean
        module_output.running_var = module.running_var
        module_output.num_batches_tracked = module.num_batches_tracked
        
        if hasattr(module, "qconfig"):
            module_output.qconfig = module.qconfig

    for name, child in module.named_children():
        module_output.add_module(name, bn2group(child))

    del module
    return module_output

net = U2NET(in_ch=3,out_ch=5)
net = bn2group(net)
# net

In [ ]:
import monai
import pytorch_lightning as pl
from pytorch_lightning.callbacks import *

class Segmentor(pl.LightningModule):
    def __init__(self, network, lossfn, metricfn, experiment_name):
        super().__init__()
        
        self.net = network
        self.lossfn = lossfn
        self.metricfn = metricfn
        self.experiment_name = experiment_name
        self.best_val_loss_epoch = np.inf            
        self.best_valid_epoch = 0
        
        if isinstance(lossfn,list) and isinstance(metricfn,list):
            assert len(lossfn) == len(metricfn)
        
    def forward(self, x):
        return self.net(x)
        
    def pipeline(self, batch, sw=False, plot=False):
        x, y = batch['image'].float(), batch['seg'].float()
        if not sw:
            yhat = self.net(x)
            loss = muti_bce_loss_fusion(yhat, y)
            yhat = yhat[0]
            metric = torch.mean(metricfn(yhat,y,))
        else:
            def predictor(x, return_idx = 0): # in case of prediction is type of list
                result = self.net(x)
                if isinstance(result, list) or isinstance(result, tuple):
                    return result[return_idx]
                else:
                    return resul
            yhat = sliding_window_inference(x, roi_size=(512,1024), sw_batch_size=2, predictor=predictor)
            loss = bce_loss(yhat,y)
            metric = torch.mean(metricfn(yhat,y,))
            # from scipy.ndimage import label, binary_closing
            # for i in range(yhat.shape[1]):
            #     for j in range(yhat.shape[1]):
            #         yhat[i,j] = binary_closing(yhat[i,j], structure=np.ones(13,13)) # closing of R-peak
    
        if plot:
            for idx in range(len(x)):
                visualize(image=x[idx].cpu(), mask1=y[idx,0].cpu().detach().numpy(), mask2=y[idx,1].cpu().detach().numpy(), mask3=y[idx,2].cpu().detach().numpy(), mask4=y[idx,3].cpu().detach().numpy(), mask5=y[idx,4].cpu().detach().numpy())
                visualize(image=x[idx].cpu(), mask1=yhat[idx,0].cpu().detach().numpy(), mask2=yhat[idx,1].cpu().detach().numpy(), mask3=yhat[idx,2].cpu().detach().numpy(), mask4=yhat[idx,3].cpu().detach().numpy(), mask5=yhat[idx,4].cpu().detach().numpy())
        return loss, metric
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.net.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=.9)
        # return optimizer
        return {'optimizer': optimizer,
                'lr_scheduler': {'scheduler': scheduler, 'monitor': 'val_loss'}}

    def training_step(self, batch, batch_idx):
        loss, metric = self.pipeline(batch, sw=False, plot=False)
        self.log('loss', loss, on_step=True, on_epoch=True, prog_bar=True,)
        if isinstance(metric,list):
            self.log('f1', metric[0], on_step=True, on_epoch=True, prog_bar=True,)
            self.log('acc', metric[1], on_step=True, on_epoch=True, prog_bar=True,)
        else:
            self.log('f1', metric, on_step=True, on_epoch=True, prog_bar=True,)            
        return loss
      
    def validation_step(self, batch, batch_idx):
        loss, metric = self.pipeline(batch, sw=True, plot=False)
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True,)
        if isinstance(metric,list):
            self.log('val_f1', metric[0], on_epoch=True, prog_bar=True,)
            self.log('val_acc', metric[1], on_epoch=True, prog_bar=True,)
        else:
            self.log('val_f1', metric, on_epoch=True, prog_bar=True,)            
        return {"val_loss":loss}
    
    def test_step(self, batch, batch_idx):
        loss, metric = self.pipeline(batch, sw=True, plot=False)
        self.log('test_loss', loss, on_step=True, on_epoch=True, prog_bar=True,)
        if isinstance(metric,list):
            self.log('test_f1', metric[0], on_epoch=True, prog_bar=True,)
            self.log('test_acc', metric[1], on_epoch=True, prog_bar=True,)
        else:
            self.log('test_f1', metric, on_epoch=True, prog_bar=True,)            
        
    def validation_epoch_end(self, outputs):
        val_losses = []
        for output in outputs:
            val_losses.append(output["val_loss"].cpu().detach().numpy())
        val_loss_epoch = np.mean(val_losses)
        # self.log('val_loss_epoch', val_loss_epoch)
        
        if val_loss_epoch < self.best_val_loss_epoch and self.current_epoch>0:
            self.best_valid_epoch = self.current_epoch
            self.best_val_loss_epoch = val_loss_epoch             
        print(
            f"current epoch: {self.current_epoch}, "
            f"current epoch val_loss: {val_loss_epoch:.4f}, "
            f"best epoch val_loss: {self.best_val_loss_epoch:.4f}, "
            f"at epoch: {self.best_valid_epoch}, " 
        )
                
import torchmetrics
lossfn = monai.losses.DiceFocalLoss(to_onehot_y=False)

# metricfn = torchmetrics.functional.dice_score
metricfn = torchmetrics.functional.dice_score
# metricfn = sklearn.metrics.f1_score

# model
experiment_name = 'vit'
model = Segmentor(network=net, lossfn=lossfn, metricfn=metricfn, experiment_name=experiment_name,)

early_stop_callback = EarlyStopping(monitor="val_loss", patience=10, verbose=False, mode="min")
checkpoint_callback = ModelCheckpoint(dirpath=f"weight/{experiment_name}/", save_top_k=1, monitor="val_loss_epoch")
trainer = pl.Trainer(gpus=-1, strategy='dp', precision=32, max_epochs=1000, callbacks=[checkpoint_callback, early_stop_callback])

In [ ]:
# model.load_from_checkpoint(network=net, lossfn=lossfn, metricfn=metricfn, experiment_name=experiment_name,checkpoint_path='weight/U2NET/epoch=9-step=229.ckpt')
model.load_from_checkpoint(network=net, lossfn=lossfn, metricfn=metricfn, experiment_name=experiment_name,checkpoint_path='weight/U2NET/epoch=18-step=436.ckpt')
model = model.cuda()

# Result

In [ ]:
# # !pip install pydicom
# # import pydicom
# # import nibabel as nib
# import cv2, skimage
# from skimage.morphology import disk
# from scipy.spatial.distance import directed_hausdorff
# from sklearn.metrics import f1_score
# import monai

# metricfn = f1_score
# metricfn = directed_hausdorff

# def mask2contour(mask):
#     mask = mask.astype(np.uint8)
#     contour = cv2.Laplacian(mask,cv2.CV_8U,ksize=3)
#     result = skimage.morphology.dilation(contour,disk(4))
#     result = skimage.morphology.erosion(result,disk(4))
#     result[result!=0] = 1
#     return result

In [ ]:
# import pydicom
# import nibabel as nib
import cv2, skimage
from skimage.morphology import disk
from scipy.spatial.distance import directed_hausdorff
from sklearn.metrics import f1_score
from monai.transforms import LabelToContour

metricfn = f1_score
metricfn = directed_hausdorff

def mask2contour(mask):
    mask = mask.astype(np.uint8)
    contour = cv2.Laplacian(mask,cv2.CV_8U,ksize=3)
    result = skimage.morphology.dilation(contour,disk(4))
    result = skimage.morphology.erosion(result,disk(4))
    result[result!=0] = 1
    return result

def postprocess(mask):
    post_process = monai.transforms.KeepLargestConnectedComponent(applied_labels=[1], connectivity=2,)
    return post_process(torch.tensor(mask.round()))

postProcess = Compose(
    [
        KeepLargestConnectedComponent(applied_labels=[1], is_onehot=True, connectivity=2),
        FillHoles(applied_labels=[1])
    ]
)

import numpy as np
from skimage.measure import label   

def getLargestCC(segmentation):
    labels = label(segmentation)
    assert( labels.max() != 0 ) # assume at least 1 CC
    largestCC = labels == np.argmax(np.bincount(labels.flat)[1:])+1
    return largestCC.astype(np.uint8)

def test(loader, plot=False):
    net.eval()
    
    xs = list()
    ys = list()
    yhats = list()
    fnames = list()
    DSCs = list()
    HDs = list()
    angle_PCA_ys = list()
    angle_PCA_yhats = list()
    angle_inertia_ys = list()
    angle_inertia_yhats = list()
    angle_ellipse_ys = list()
    angle_ellipse_yhats = list()
    
    for idx, batch in tqdm(enumerate(loader)):
        x, y_seg, fname = batch['image'], batch['seg'], batch['fname']
        print(fname)
        xs.append(x)
        ys.append(y_seg)
        fnames.append(fname)
        
        x = x.cuda()
        
        with torch.no_grad():
            yhat = net(x)
            yhat = yhat[0] # only for U2NET
            
            def predictor(x, return_idx = 0): # in case of prediction is type of list
                result = model(x)
                if isinstance(result, list) or isinstance(result, tuple):
                    return result[return_idx]
                else:
                    return result
                
            # print(resize_toNet(x).shape)
            # plt.imshow(resize_toNet(x)[0,0].cpu().detach())
            # plt.show()
                
            # yhat = sliding_window_inference(resize_toNet(x), roi_size=(512,1024), sw_batch_size=2, predictor=predictor)
            yhat = sliding_window_inference(x, roi_size=(512, 1024), sw_batch_size=2, predictor=predictor)
            # yhat = resize_toOriginal(yhat)
            # yhat = pad(yhat)
            # yhat = yhat.cpu().detach().numpy()
            # yhat[0] = postProcess(yhat[0].round())
            yhat = yhat.cpu().detach().numpy()
            # yhat = postprocess(yhat)
            
            temp_DSC = np.zeros((5))
            temp_HD = np.zeros((5))
            temp_angle_PCA_y = np.zeros((5))
            temp_angle_PCA_yhat = np.zeros((5))
            temp_angle_inertia_y = np.zeros((5))
            temp_angle_inertia_yhat = np.zeros((5))
            temp_angle_ellipse_y = np.zeros((5))
            temp_angle_ellipse_yhat = np.zeros((5))
            
            for idx_class in range(yhat.shape[1]):
                # yhat = getLargestCC(yhat)
                yhat[0][idx_class] = getLargestCC(yhat[0][idx_class].round())
                DSC = f1_score(y_seg[0][idx_class].flatten(), yhat[0][idx_class].flatten())
                HD = directed_hausdorff(y_seg[0][idx_class], yhat[0][idx_class])[0]
                
                temp_DSC[idx_class] = DSC
                temp_HD[idx_class] = HD
    
                head = head_direction(y_seg[0][0])
                major = True if idx_class <4 else False     
                clockwise_plus = True if idx_class==0 else False
            
                x0,y0,angle = PCA_axis(y_seg[0][idx_class].numpy(),major)
                temp_angle_PCA_y[idx_class] = refine_degree(angle,head,clockwise_plus=clockwise_plus)

                x0,y0,angle = PCA_axis(yhat[0][idx_class].round(),major)
                temp_angle_PCA_yhat[idx_class] = refine_degree(angle,head,clockwise_plus=clockwise_plus)
                                                         
                x0,y0,angle = ellipse_axis(y_seg[0][idx_class].numpy(),major)
                temp_angle_ellipse_y[idx_class] = refine_degree(angle,head,clockwise_plus=clockwise_plus)
    
                x0,y0,angle = ellipse_axis(yhat[0][idx_class].round(),major)
                temp_angle_ellipse_yhat[idx_class] = refine_degree(angle,head,clockwise_plus=clockwise_plus)
                                                         
                x0,y0,angle = inertia_axis(y_seg[0][idx_class].numpy(),major)
                temp_angle_inertia_y[idx_class] = refine_degree(angle,head,clockwise_plus=clockwise_plus)
                
                x0,y0,angle = inertia_axis(yhat[0][idx_class].round(),major)
                temp_angle_inertia_yhat[idx_class] = refine_degree(angle,head,clockwise_plus=clockwise_plus)
            
            if plot:
                visualize(
                    x=x[0].cpu().detach(), 
                    y1 =y_seg[0][0].round(),
                    y2 =y_seg[0][1].round(),
                    y3 =y_seg[0][2].round(),
                    y4 =y_seg[0][3].round(),
                    y5 =y_seg[0][4].round(),
                )
                visualize(
                    x=x[0].cpu().detach(), 
                    yhat1 =yhat[0][0].round(),
                    yhat2 =yhat[0][1].round(),
                    yhat3 =yhat[0][2].round(),
                    yhat4 =yhat[0][3].round(),
                    yhat5 =yhat[0][4].round(),
                )
                
            yhats.append(yhat)
            DSCs.append(temp_DSC)
            HDs.append(temp_HD)
            angle_PCA_ys.append(temp_angle_PCA_y)
            angle_PCA_yhats.append(temp_angle_PCA_yhat)
            angle_inertia_ys.append(temp_angle_inertia_y)
            angle_inertia_yhats.append(temp_angle_inertia_yhat)
            angle_ellipse_ys.append(temp_angle_ellipse_y)
            angle_ellipse_yhats.append(temp_angle_ellipse_yhat)
            
    DSCs = np.array(DSCs)
    HDs = np.array(HDs)
    angle_PCA_ys = np.array(angle_PCA_ys)
    angle_PCA_yhats = np.array(angle_PCA_yhats)
    angle_inertia_ys = np.array(angle_inertia_ys)
    angle_inertia_yhats = np.array(angle_inertia_yhats)
    angle_ellipse_ys = np.array(angle_ellipse_ys)
    angle_ellipse_yhats = np.array(angle_ellipse_yhats)
    
    return {'xs':xs, 'ys':ys, 'yhats':yhats, 'fnames':fnames, 'DSCs':DSCs, 'HDs':HDs, 
            'angle_PCA_ys':angle_PCA_ys, 'angle_PCA_yhats':angle_PCA_yhats,
            'angle_inertia_ys':angle_inertia_ys, 'angle_inertia_yhats':angle_inertia_yhats,
            'angle_ellipse_ys':angle_ellipse_ys, 'angle_ellipse_yhats':angle_ellipse_yhats}

In [ ]:
# # train 각도만 추출
# result = test(train_loader)
# # STAGE='ActiveLearning'
# # STAGE='NonActiveLearning'

# xs = result['xs']
# ys = result['ys']
# fnames = result['fnames']

# angle_PCA_ys = result['angle_PCA_ys']
# angle_inertia_ys = result['angle_inertia_ys']
# angle_ellipse_ys = result['angle_ellipse_ys']

# df = pd.DataFrame([fnames,angle_PCA_ys[:,0],angle_PCA_ys[:,1],angle_PCA_ys[:,2],angle_PCA_ys[:,3],angle_PCA_ys[:,4],
#                    angle_inertia_ys[:,0],angle_inertia_ys[:,1],angle_inertia_ys[:,2],angle_inertia_ys[:,3],angle_inertia_ys[:,4],
#                    angle_ellipse_ys[:,0],angle_ellipse_ys[:,1],angle_ellipse_ys[:,2],angle_ellipse_ys[:,3],angle_ellipse_ys[:,4]],
#                   ['fnames',STAGE+'_angle_PCA_y_1',STAGE+'_angle_PCA_y_2',STAGE+'_angle_PCA_y_3',STAGE+'_angle_PCA_y_4',STAGE+'_angle_PCA_y_5',
#                    STAGE+'_angle_inertia_y_1',STAGE+'_angle_inertia_y_2',STAGE+'_angle_inertia_y_3',STAGE+'_angle_inertia_y_4',STAGE+'_angle_inertia_y_5',
#                    STAGE+'_angle_ellipse_y_1',STAGE+'_angle_ellipse_y_2',STAGE+'_angle_ellipse_y_3',STAGE+'_angle_ellipse_y_4',STAGE+'_angle_ellipse_y_5'])
# df = df.T
# df
# # df.to_csv(STAGE+'_metrics_train.csv',index=False)
# df.to_csv('20220428_DrRyu_test.csv',index=False)

In [ ]:
# from tqdm import trange

# # Afiles = natsort.natsorted(glob.glob('ExternalObserver/SMR/x_test/*.png'))
# # Afiles = natsort.natsorted(glob.glob('ActiveLearning/4th/x_test/*.png'))
# Afiles = natsort.natsorted(glob.glob('ActiveLearning/4th/x_test/*.png'))[:50] + natsort.natsorted(glob.glob('ExternalObserver/SMR/x_test/*.png'))

# Gfiles = natsort.natsorted(glob.glob('ExternalObserver/GP1/x_test/*.png'))
# gfiles = natsort.natsorted(glob.glob('ExternalObserver/GP1/y_test/*.npy'))

# # Gfiles = natsort.natsorted(glob.glob('ExternalObserver/GP2/x_test/*.png'))
# # gfiles = natsort.natsorted(glob.glob('ExternalObserver/GP2/y_test/*.npy'))

# # Gfiles = natsort.natsorted(glob.glob('ExternalObserver/OS/x_test/*.png'))
# # gfiles = natsort.natsorted(glob.glob('ExternalObserver/OS/y_test/*.npy'))

# print(len(Afiles), len(Gfiles))
# for idx in trange(100):
#     A = cv2.imread(Afiles[idx],0)
#     G = cv2.imread(Gfiles[idx],0)
#     h = G.shape[0] - A.shape[0]
#     w = A.shape[1] - G.shape[1]
#     print(A.shape,G.shape,h,w)
    
#     # file = Gfiles[idx]
#     # G = G[h:].astype(np.uint8)
#     # cv2.imwrite(file,G)
    
# #     file = gfiles[idx]
# #     g = np.load(file)
# #     g = g[h:].astype(np.uint8)
# #     np.save(file,g)
 
#     # print(A.shape, G.shape, g.shape)

In [ ]:
# # iterator = iter(test_loader)
# # iteratorGP1 = iter(testGP1_loader)
# # iteratorGP2 = iter(testGP2_loader)
# # iteratorOS = iter(testOS_loader)

def DSC_HD(loaderGT,loaderHAT,STAGE):
    
    DSCs = list()
    HDs = list()
    fnames = list()
    
    iterator = iter(loaderGT)
    iterator_ = iter(loaderHAT)
    
    for idx in trange(len(loaderGT)):        
        batch = iterator.next()  
        batch_ = iterator_.next()  

        x, y, fname = batch['image'], batch['seg'], batch['fname']
        x_, y_, fname_ = batch_['image'], batch_['seg'], batch_['fname']
        
        temp_DSC = np.zeros((5))
        temp_HD = np.zeros((5))

        for idx_class in range(y.shape[1]):
            # print(idx, y.shape,y_.shape, torch.unique(y[0][idx_class]),torch.unique(y_[0][idx_class]))
            # plt.imshow(y[0][idx_class])
            # plt.show()
            # plt.imshow(y_[0][idx_class])
            # plt.show()
            DSC = f1_score(y[0][idx_class].flatten(), y_[0][idx_class].flatten())
            HD = directed_hausdorff(y[0][idx_class], y_[0][idx_class])[0]
            
            temp_DSC[idx_class] = DSC
            temp_HD[idx_class] = HD
        
        DSCs.append(temp_DSC)
        HDs.append(temp_HD)
        fnames.append(fname)
    
    DSCs = np.array(DSCs)
    HDs = np.array(HDs)
    df = pd.DataFrame([fnames,DSCs[:,0],DSCs[:,1],DSCs[:,2],DSCs[:,3],DSCs[:,4],HDs[:,0],HDs[:,1],HDs[:,2],HDs[:,3],HDs[:,4]],
                      ['fnames',STAGE+'_DSC_1',STAGE+'_DSC_2',STAGE+'_DSC_3',STAGE+'_DSC_4',STAGE+'_DSC_5',STAGE+'_HD_1',STAGE+'_HD_2',STAGE+'_HD_3',STAGE+'_HD_4',STAGE+'_HD_5'])
    df = df.T
        
    return df

STAGE = 'OS'
df = DSC_HD(test_loader,testOS_loader,STAGE)
df.to_csv(f'20220502_{STAGE}_test.csv',index=False)

In [ ]:
result = test(test_loader, plot=True)

STAGE = 'DR_Ryu'
# STAGE = 'GP1'
# STAGE = 'GP2'
# STAGE = 'OS'

xs = result['xs']
ys = result['ys']
yhats = result['yhats']
fnames = result['fnames']
DSCs = result['DSCs']
HDs = result['HDs']
                                                            
angle_PCA_ys = result['angle_PCA_ys']
angle_PCA_yhats = result['angle_PCA_yhats']
angle_inertia_ys = result['angle_inertia_ys']
angle_inertia_yhats = result['angle_inertia_yhats']
angle_ellipse_ys = result['angle_ellipse_ys']
angle_ellipse_yhats = result['angle_ellipse_yhats']
# np.save('{}/result.npy'.format(STAGE), result)

print(len(xs),len(ys),len(yhats),len(fnames),len(DSCs),len(HDs))

for idx in range(DSCs.shape[1]):
    print('class {:2d}, DSC {:3.3f}, HD {:3.3f}'.format(idx+1,np.mean(DSCs[:,idx]),np.mean(HDs[:,idx])))

df = pd.DataFrame([fnames,DSCs[:,0],DSCs[:,1],DSCs[:,2],DSCs[:,3],DSCs[:,4],HDs[:,0],HDs[:,1],HDs[:,2],HDs[:,3],HDs[:,4],angle_PCA_ys[:,0],angle_PCA_ys[:,1],angle_PCA_ys[:,2],angle_PCA_ys[:,3],angle_PCA_ys[:,4],angle_PCA_yhats[:,0],angle_PCA_yhats[:,1],angle_PCA_yhats[:,2],angle_PCA_yhats[:,3],angle_PCA_yhats[:,4],angle_inertia_ys[:,0],angle_inertia_ys[:,1],angle_inertia_ys[:,2],angle_inertia_ys[:,3],angle_inertia_ys[:,4],angle_inertia_yhats[:,0],angle_inertia_yhats[:,1],angle_inertia_yhats[:,2],angle_inertia_yhats[:,3],angle_inertia_yhats[:,4],angle_ellipse_ys[:,0],angle_ellipse_ys[:,1],angle_ellipse_ys[:,2],angle_ellipse_ys[:,3],angle_ellipse_ys[:,4],angle_ellipse_yhats[:,0],angle_ellipse_yhats[:,1],angle_ellipse_yhats[:,2],angle_ellipse_yhats[:,3],angle_ellipse_yhats[:,4]],['fnames',STAGE+'_DSC_1',STAGE+'_DSC_2',STAGE+'_DSC_3',STAGE+'_DSC_4',STAGE+'_DSC_5',STAGE+'_HD_1',STAGE+'_HD_2',STAGE+'_HD_3',STAGE+'_HD_4',STAGE+'_HD_5',STAGE+'_angle_PCA_y_1',STAGE+'_angle_PCA_y_2',STAGE+'_angle_PCA_y_3',STAGE+'_angle_PCA_y_4',STAGE+'_angle_PCA_y_5',STAGE+'_angle_PCA_yhat_1',STAGE+'_angle_PCA_yhat_2',STAGE+'_angle_PCA_yhat_3',STAGE+'_angle_PCA_yhat_4',STAGE+'_angle_PCA_yhat_5',STAGE+'_angle_inertia_y_1',STAGE+'_angle_inertia_y_2',STAGE+'_angle_inertia_y_3',STAGE+'_angle_inertia_y_4',STAGE+'_angle_inertia_y_5',STAGE+'_angle_inertia_yhat_1',STAGE+'_angle_inertia_yhat_2',STAGE+'_angle_inertia_yhat_3',STAGE+'_angle_inertia_yhat_4',STAGE+'_angle_inertia_yhat_5',STAGE+'_angle_ellipse_y_1',STAGE+'_angle_ellipse_y_2',STAGE+'_angle_ellipse_y_3',STAGE+'_angle_ellipse_y_4',STAGE+'_angle_ellipse_y_5',STAGE+'_angle_ellipse_yhat_1',STAGE+'_angle_ellipse_yhat_2',STAGE+'_angle_ellipse_yhat_3',STAGE+'_angle_ellipse_yhat_4',STAGE+'_angle_ellipse_yhat_5'])
df = df.T
# df.to_csv('{}/metrics.csv'.format(STAGE),index=False)
# df.to_csv('20220428_DrRyu_test.csv',index=False)

# df.to_csv(f'20220428_{STAGE}_test.csv',index=False)

In [ ]:
result = test(etest_loader,plot=True)

# STAGE = 'DR_Ryu_LERA'
STAGE = 'GP1_LERA'

xs = result['xs']
ys = result['ys']
yhats = result['yhats']
fnames = result['fnames']
DSCs = result['DSCs']
HDs = result['HDs']
                                                            
angle_PCA_ys = result['angle_PCA_ys']
angle_PCA_yhats = result['angle_PCA_yhats']
angle_inertia_ys = result['angle_inertia_ys']
angle_inertia_yhats = result['angle_inertia_yhats']
angle_ellipse_ys = result['angle_ellipse_ys']
angle_ellipse_yhats = result['angle_ellipse_yhats']
# np.save('{}/result.npy'.format(STAGE), result)

print(len(xs),len(ys),len(yhats),len(fnames),len(DSCs),len(HDs))

for idx in range(DSCs.shape[1]):
    print('class {:2d}, DSC {:3.3f}, HD {:3.3f}'.format(idx+1,np.mean(DSCs[:,idx]),np.mean(HDs[:,idx])))

df = pd.DataFrame([fnames,DSCs[:,0],DSCs[:,1],DSCs[:,2],DSCs[:,3],DSCs[:,4],HDs[:,0],HDs[:,1],HDs[:,2],HDs[:,3],HDs[:,4],angle_PCA_ys[:,0],angle_PCA_ys[:,1],angle_PCA_ys[:,2],angle_PCA_ys[:,3],angle_PCA_ys[:,4],angle_PCA_yhats[:,0],angle_PCA_yhats[:,1],angle_PCA_yhats[:,2],angle_PCA_yhats[:,3],angle_PCA_yhats[:,4],angle_inertia_ys[:,0],angle_inertia_ys[:,1],angle_inertia_ys[:,2],angle_inertia_ys[:,3],angle_inertia_ys[:,4],angle_inertia_yhats[:,0],angle_inertia_yhats[:,1],angle_inertia_yhats[:,2],angle_inertia_yhats[:,3],angle_inertia_yhats[:,4],angle_ellipse_ys[:,0],angle_ellipse_ys[:,1],angle_ellipse_ys[:,2],angle_ellipse_ys[:,3],angle_ellipse_ys[:,4],angle_ellipse_yhats[:,0],angle_ellipse_yhats[:,1],angle_ellipse_yhats[:,2],angle_ellipse_yhats[:,3],angle_ellipse_yhats[:,4]],['fnames',STAGE+'_DSC_1',STAGE+'_DSC_2',STAGE+'_DSC_3',STAGE+'_DSC_4',STAGE+'_DSC_5',STAGE+'_HD_1',STAGE+'_HD_2',STAGE+'_HD_3',STAGE+'_HD_4',STAGE+'_HD_5',STAGE+'_angle_PCA_y_1',STAGE+'_angle_PCA_y_2',STAGE+'_angle_PCA_y_3',STAGE+'_angle_PCA_y_4',STAGE+'_angle_PCA_y_5',STAGE+'_angle_PCA_yhat_1',STAGE+'_angle_PCA_yhat_2',STAGE+'_angle_PCA_yhat_3',STAGE+'_angle_PCA_yhat_4',STAGE+'_angle_PCA_yhat_5',STAGE+'_angle_inertia_y_1',STAGE+'_angle_inertia_y_2',STAGE+'_angle_inertia_y_3',STAGE+'_angle_inertia_y_4',STAGE+'_angle_inertia_y_5',STAGE+'_angle_inertia_yhat_1',STAGE+'_angle_inertia_yhat_2',STAGE+'_angle_inertia_yhat_3',STAGE+'_angle_inertia_yhat_4',STAGE+'_angle_inertia_yhat_5',STAGE+'_angle_ellipse_y_1',STAGE+'_angle_ellipse_y_2',STAGE+'_angle_ellipse_y_3',STAGE+'_angle_ellipse_y_4',STAGE+'_angle_ellipse_y_5',STAGE+'_angle_ellipse_yhat_1',STAGE+'_angle_ellipse_yhat_2',STAGE+'_angle_ellipse_yhat_3',STAGE+'_angle_ellipse_yhat_4',STAGE+'_angle_ellipse_yhat_5'])
df = df.T
# df.to_csv(f'20220428_{STAGE}_test.csv',index=False)

In [ ]:
result = test(etest2_loader,plot=True)
STAGE = 'DR_Ryu_AMC'

xs = result['xs']
ys = result['ys']
yhats = result['yhats']
fnames = result['fnames']
DSCs = result['DSCs']
HDs = result['HDs']
                                                            
angle_PCA_ys = result['angle_PCA_ys']
angle_PCA_yhats = result['angle_PCA_yhats']
angle_inertia_ys = result['angle_inertia_ys']
angle_inertia_yhats = result['angle_inertia_yhats']
angle_ellipse_ys = result['angle_ellipse_ys']
angle_ellipse_yhats = result['angle_ellipse_yhats']
# np.save('{}/result.npy'.format(STAGE), result)

print(len(xs),len(ys),len(yhats),len(fnames),len(DSCs),len(HDs))

for idx in range(DSCs.shape[1]):
    print('class {:2d}, DSC {:3.3f}, HD {:3.3f}'.format(idx+1,np.mean(DSCs[:,idx]),np.mean(HDs[:,idx])))

df = pd.DataFrame([fnames,DSCs[:,0],DSCs[:,1],DSCs[:,2],DSCs[:,3],DSCs[:,4],HDs[:,0],HDs[:,1],HDs[:,2],HDs[:,3],HDs[:,4],angle_PCA_ys[:,0],angle_PCA_ys[:,1],angle_PCA_ys[:,2],angle_PCA_ys[:,3],angle_PCA_ys[:,4],angle_PCA_yhats[:,0],angle_PCA_yhats[:,1],angle_PCA_yhats[:,2],angle_PCA_yhats[:,3],angle_PCA_yhats[:,4],angle_inertia_ys[:,0],angle_inertia_ys[:,1],angle_inertia_ys[:,2],angle_inertia_ys[:,3],angle_inertia_ys[:,4],angle_inertia_yhats[:,0],angle_inertia_yhats[:,1],angle_inertia_yhats[:,2],angle_inertia_yhats[:,3],angle_inertia_yhats[:,4],angle_ellipse_ys[:,0],angle_ellipse_ys[:,1],angle_ellipse_ys[:,2],angle_ellipse_ys[:,3],angle_ellipse_ys[:,4],angle_ellipse_yhats[:,0],angle_ellipse_yhats[:,1],angle_ellipse_yhats[:,2],angle_ellipse_yhats[:,3],angle_ellipse_yhats[:,4]],['fnames',STAGE+'_DSC_1',STAGE+'_DSC_2',STAGE+'_DSC_3',STAGE+'_DSC_4',STAGE+'_DSC_5',STAGE+'_HD_1',STAGE+'_HD_2',STAGE+'_HD_3',STAGE+'_HD_4',STAGE+'_HD_5',STAGE+'_angle_PCA_y_1',STAGE+'_angle_PCA_y_2',STAGE+'_angle_PCA_y_3',STAGE+'_angle_PCA_y_4',STAGE+'_angle_PCA_y_5',STAGE+'_angle_PCA_yhat_1',STAGE+'_angle_PCA_yhat_2',STAGE+'_angle_PCA_yhat_3',STAGE+'_angle_PCA_yhat_4',STAGE+'_angle_PCA_yhat_5',STAGE+'_angle_inertia_y_1',STAGE+'_angle_inertia_y_2',STAGE+'_angle_inertia_y_3',STAGE+'_angle_inertia_y_4',STAGE+'_angle_inertia_y_5',STAGE+'_angle_inertia_yhat_1',STAGE+'_angle_inertia_yhat_2',STAGE+'_angle_inertia_yhat_3',STAGE+'_angle_inertia_yhat_4',STAGE+'_angle_inertia_yhat_5',STAGE+'_angle_ellipse_y_1',STAGE+'_angle_ellipse_y_2',STAGE+'_angle_ellipse_y_3',STAGE+'_angle_ellipse_y_4',STAGE+'_angle_ellipse_y_5',STAGE+'_angle_ellipse_yhat_1',STAGE+'_angle_ellipse_yhat_2',STAGE+'_angle_ellipse_yhat_3',STAGE+'_angle_ellipse_yhat_4',STAGE+'_angle_ellipse_yhat_5'])
df = df.T
# df.to_csv('{}/metrics.csv'.format(STAGE),index=False)
# df.to_csv('20220428_DrRyu_test.csv',index=False)
# df.to_csv(f'20220428_{STAGE}_test.csv',index=False)

In [ ]:
for idx in range(len(xs)):
    print(idx, fnames[idx], DSCs[idx], HDs[idx])
    visualize(
        x=xs[idx][0], 
        y1=ys[idx][0][0],
        y2=ys[idx][0][1],
        y3=ys[idx][0][2],
        y4=ys[idx][0][3],
        y5=ys[idx][0][4], 
    )
    visualize(
        x=xs[idx][0], 
        yhat1 =yhats[idx][0][0].round(),
        yhat2 =yhats[idx][0][1].round(),
        yhat3 =yhats[idx][0][2].round(),
        yhat4 =yhats[idx][0][3].round(),
        yhat5 =yhats[idx][0][4].round(),
    )

#     visualize(
#         x=xs[idx][0], 
#         yhat1_contour = mask2contour(yhats[idx][0][0].round()),
#         yhat2_contour = mask2contour(yhats[idx][0][1].round()),
#         yhat3_contour = mask2contour(yhats[idx][0][2].round()),
#         yhat4_contour = mask2contour(yhats[idx][0][3].round()),
#         yhat5_contour = mask2contour(yhats[idx][0][4].round()),
#     )

In [ ]:


# ACTIVE LEARNING
# 1st DICE
# class  1, DSC 0.978, HD 4.811
# class  2, DSC 0.968, HD 3.895
# class  3, DSC 0.921, HD 6.931
# class  4, DSC 0.934, HD 6.170
# class  5, DSC 0.919, HD 3.586

# 2nd DICE
# class  1, DSC 0.981, HD 4.662
# class  2, DSC 0.976, HD 3.885
# class  3, DSC 0.969, HD 4.518
# class  4, DSC 0.954, HD 5.234
# class  5, DSC 0.928, HD 3.575

# 3rd DICE
# class  1, DSC 0.986, HD 4.392
# class  2, DSC 0.978, HD 3.703
# class  3, DSC 0.975, HD 4.536
# class  4, DSC 0.960, HD 5.145
# class  5, DSC 0.932, HD 3.469

# 4th DICE
# class  1, DSC 0.986, HD 4.434
# class  2, DSC 0.978, HD 3.783
# class  3, DSC 0.976, HD 4.383
# class  4, DSC 0.964, HD 5.135
# class  5, DSC 0.931, HD 3.518

# NONACTIVE LEARNING
# 1st DICE
# class  1, DSC 0.978, HD 4.811
# class  2, DSC 0.968, HD 3.895
# class  3, DSC 0.921, HD 6.931
# class  4, DSC 0.934, HD 6.170
# class  5, DSC 0.919, HD 3.586

# 2nd DICE
# class  1, DSC 0.985, HD 4.503
# class  2, DSC 0.977, HD 3.919
# class  3, DSC 0.974, HD 4.317
# class  4, DSC 0.950, HD 5.492
# class  5, DSC 0.932, HD 3.520

# 3rd DICE
# class  1, DSC 0.984, HD 4.536
# class  2, DSC 0.976, HD 3.787
# class  3, DSC 0.973, HD 4.421
# class  4, DSC 0.955, HD 5.318
# class  5, DSC 0.928, HD 3.630

# 4th DICE
# class  1, DSC 0.986, HD 4.434
# class  2, DSC 0.978, HD 3.783
# class  3, DSC 0.976, HD 4.383
# class  4, DSC 0.964, HD 5.135
# class  5, DSC 0.931, HD 3.518

In [ ]:
for idx in range(len(xs)):
    print(idx, fnames[idx], DSCs[idx], HDs[idx])
    visualize(
        x=xs[idx][0], 
        y1=ys[idx][0][0],
        y2=ys[idx][0][1],
        y3=ys[idx][0][2],
        y4=ys[idx][0][3],
        y5=ys[idx][0][4], 
    )
    visualize(
        x=xs[idx][0], 
        yhat1 =yhats[idx][0][0].round(),
        yhat2 =yhats[idx][0][1].round(),
        yhat3 =yhats[idx][0][2].round(),
        yhat4 =yhats[idx][0][3].round(),
        yhat5 =yhats[idx][0][4].round(),
    )

#     visualize(
#         x=xs[idx][0], 
#         yhat1_contour = mask2contour(yhats[idx][0][0].round()),
#         yhat2_contour = mask2contour(yhats[idx][0][1].round()),
#         yhat3_contour = mask2contour(yhats[idx][0][2].round()),
#         yhat4_contour = mask2contour(yhats[idx][0][3].round()),
#         yhat5_contour = mask2contour(yhats[idx][0][4].round()),
#     )

In [ ]:
from skimage.measure import label, regionprops, regionprops_table
import math

def mask2contour(mask):
    mask = mask.astype(np.uint8)
    contour = cv2.Laplacian(mask,cv2.CV_8U,ksize=7)
    result = skimage.morphology.dilation(contour,disk(4))
    result = skimage.morphology.erosion(result,disk(4))
    result[result!=0] = 1
    return result

# inertia axis
def visualize(**images):
    """Plot images in one row."""
    angles = list()
    
    n = len(images)
    plt.figure(figsize=(32, 16))
    for i, (name, image) in enumerate(images.items()):
        if i==0 and image.shape[0] == 3:
            image_main = image.permute(1,2,0).numpy()#.int()
        if torch.is_tensor(image) and image.shape[0] == 3:
            image = image.permute(1,2,0).numpy()#.int()
        else:
            try:
                image = image.numpy()
            except:
                pass
            
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image_main,cmap='gray')
        if i>=1:
            prop = inertia_axis(image)
            coord_x = int(prop[0])
            coord_y = int(prop[1])
            angle = prop[2]
            offset= 200
            image[coord_y-5:coord_y+5,coord_x-5:coord_x+5] = 0
            image = cv2.line(image,(coord_x-offset, coord_y-int(offset*np.tan(angle))),(coord_x+offset, coord_y+int(offset*np.tan(angle))),(0,255,255),5)
            angle = np.degrees(angle)*(-1)
            angles.append(angle)
            plt.title(' '.join(name.split('_')).title())
            plt.imshow(image,cmap='gray',alpha=0.6)
            plt.tight_layout()
#             plt.plot(coord_x, coord_y, '.g', markersize=15)
    plt.show()
    return angles

for idx in range(len(xs)):
    print(idx, fnames[idx], DSCs[idx], HDs[idx])
    visualize(
        x=xs[idx][0], 
        yhat1_contour = mask2contour(yhats[idx][0][0].round()),
        yhat2_contour = mask2contour(yhats[idx][0][1].round()),
        yhat3_contour = mask2contour(yhats[idx][0][2].round()),
        yhat4_contour = mask2contour(yhats[idx][0][3].round()),
        yhat5_contour = mask2contour(yhats[idx][0][4].round()),
    )

In [ ]:
a = label2Contour(torch.tensor(np.expand_dims(yhats[idx][0][0].round(),0)))
a.shape

In [ ]:
from monai.transforms import LabelToContour
label2Contour = LabelToContour()
for idx in range(len(xs)):
    print(idx, fnames[idx], DSCs[idx], HDs[idx])
    visualize(
        x=xs[idx][0], 
        yhat1_contour = label2Contour(yhats[idx][0][0].round()),
        yhat2_contour = label2Contour(yhats[idx][0][1].round()),
        yhat3_contour = label2Contour(yhats[idx][0][2].round()),
        yhat4_contour = label2Contour(yhats[idx][0][3].round()),
        yhat5_contour = label2Contour(yhats[idx][0][4].round()),
    )

# Etest

In [ ]:
import pydicom
import nibabel as nib
import cv2, skimage
from skimage.morphology import disk
from scipy.spatial.distance import directed_hausdorff
from sklearn.metrics import f1_score

metricfn = f1_score
metricfn = directed_hausdorff

def mask2contour(mask):
    mask = mask.astype(np.uint8)
    contour = cv2.Laplacian(mask,cv2.CV_8U,ksize=3)
    result = skimage.morphology.dilation(contour,disk(4))
    result = skimage.morphology.erosion(result,disk(4))
    result[result!=0] = 1
    return result

def test():
    model.eval()
    
    xs = list()
    ys = list()
    yhats = list()
    fnames = list()
    DSCs = list()
    HDs = list()
    
    for idx, batch in tqdm.tqdm(enumerate(etest_loader)):
        x, y_seg,fname = batch['x'],batch['y_seg'], batch['fname']
        
        xs.append(x)
        ys.append(y_seg)
        fnames.append(fname)
        
        x, y_seg = x.cuda(), y_seg.cuda()
        
        with torch.no_grad():
            yhat = model(x)            
            yhats.append(yhat.cpu().detach().numpy())

            temp_DSC = np.zeros((5))
            temp_HD = np.zeros((5))
            for idx_class in range(yhat.shape[1]):
                DSC = f1_score(ys[idx][0][idx_class].flatten(), yhats[idx][0][idx_class].round().flatten())
                HD = directed_hausdorff(ys[idx][0][idx_class], yhats[idx][0][idx_class].round())[0]
                
                temp_DSC[idx_class] = DSC
                temp_HD[idx_class] = HD
            DSCs.append(temp_DSC)
            HDs.append(temp_HD)
            
    DSCs = np.array(DSCs)
    HDs = np.array(HDs)
    
    return {'xs':xs,'ys':ys, 'yhats':yhats, 'fnames':fnames, 'DSCs':DSCs, 'HDs':HDs }

result = test()

xs= result['xs']
ys = result['ys']
yhats = result['yhats']
fnames = result['fnames']
DSCs = result['DSCs']
HDs = result['HDs']
# np.save('result_'+STAGE+'.npy', result)

print(len(xs),len(ys),len(yhats),len(fnames),len(DSCs),len(HDs))

In [ ]:
for idx in range(len(xs)):
    print(idx, fnames[idx], DSCs[idx], HDs[idx])
    visualize(
        x=xs[idx][0], 
        y1=ys[idx][0][0],
        y2=ys[idx][0][1],
        y3=ys[idx][0][2],
        y4=ys[idx][0][3],
        y5=ys[idx][0][4], 
    )
    visualize(
        x=xs[idx][0], 
        yhat1 =yhats[idx][0][0].round(),
        yhat2 =yhats[idx][0][1].round(),
        yhat3 =yhats[idx][0][2].round(),
        yhat4 =yhats[idx][0][3].round(),
        yhat5 =yhats[idx][0][4].round(),
    )

    visualize(
        x=xs[idx][0], 
        yhat1_contour = mask2contour(yhats[idx][0][0].round()),
        yhat2_contour = mask2contour(yhats[idx][0][1].round()),
        yhat3_contour = mask2contour(yhats[idx][0][2].round()),
        yhat4_contour = mask2contour(yhats[idx][0][3].round()),
        yhat5_contour = mask2contour(yhats[idx][0][4].round()),
    )

# Inference

In [ ]:
def test():
    model.eval()
    
    xs = list()
    ys = list()
    yhats = list()
    fnames = list()    
    DSCs = list()
    HDs = list()

    
    for idx, batch in tqdm.tqdm(enumerate(inference_loader)):
        x, y_seg,fname = batch['x'],batch['y_seg'], batch['fname']
        
        xs.append(x)
        ys.append(y_seg)
        fnames.append(fname)
        
        x, y_seg = x.cuda(), y_seg.cuda()
        
        with torch.no_grad():
            yhat = model(x)            
            yhats.append(yhat.cpu().detach().numpy())

            temp_DSC = np.zeros((5))
            temp_HD = np.zeros((5))
            for idx_class in range(yhat.shape[1]):
                DSC = f1_score(ys[idx][0][idx_class].flatten(), yhats[idx][0][idx_class].round().flatten())
                HD = directed_hausdorff(ys[idx][0][idx_class], yhats[idx][0][idx_class].round())[0]
                
                temp_DSC[idx_class] = DSC
                temp_HD[idx_class] = HD
            DSCs.append(temp_DSC)
            HDs.append(temp_HD)
            
    DSCs = np.array(DSCs)
    HDs = np.array(HDs)
    
    return {'xs':xs,'ys':ys, 'yhats':yhats, 'fnames':fnames, 'DSCs':DSCs, 'HDs':HDs }

result = test()

xs= result['xs']
ys = result['ys']
yhats = result['yhats']
fnames = result['fnames']
DSCs = result['DSCs']
HDs = result['HDs']

print(len(xs),len(ys),len(yhats),len(fnames),len(DSCs),len(HDs))

In [ ]:
y_path_tosave = STAGE+'/yhat_inference/'
for idx in trange(len(yhats)):
    for idx_class in range(yhats[idx].shape[1]):
        print(STAGE+'/DCM_inference/'+fnames[idx][0].split('/')[-1].split('.')[0]+'*')
        dcm = glob.glob(STAGE+'/DCM_inference/'+fnames[idx][0].split('/')[-1].split('.')[0]+'*')[0]
        dcm = pydicom.dcmread(dcm)
        
        def pad_augmentation():
            return albu.Compose([albu.PadIfNeeded(dcm.Rows-700, dcm.Columns, border_mode=cv2.BORDER_CONSTANT, value=0, always_apply=True)])
        
        pad = pad_augmentation()
        
        affine = np.array([[dcm.PixelSpacing[0],0,0,0],[0,dcm.PixelSpacing[1],0,0],[0,0,1,0],[0,0,0,1]])
        yhat_contour = mask2contour(yhats[idx][0][idx_class].round())
        blank_upper = np.zeros((700,dcm.Columns))
        target_height = dcm.Rows - 700
        target_width = dcm.Columns
#         yhat_contour = cv2.resize(yhat_contour,(2048,target_height),cv2.INTER_CUBIC)
        yhat_contour = cv2.resize(yhat_contour,(2048,1024),cv2.INTER_CUBIC)
        sample = pad(image=yhat_contour, mask=yhat_contour)
        _, yhat_contour = sample['image'], sample['mask']
        print(blank_upper.shape,yhat_contour.shape)

        result = np.vstack([blank_upper,yhat_contour],)
        result = np.transpose(result)
        print(idx,fnames[idx],result.shape)
        result = nib.Nifti1Image(result.astype(np.uint16), affine)
        
        os.makedirs(y_path_tosave, mode=0o777, exist_ok=True)
        nib.save(result,STAGE+'/yhat_inference/'+fnames[idx][0].split('/')[-1].split('.')[0]+'_'+str(idx_class+1)+'.nii.gz')

In [ ]:
# target_width,target_height

In [ ]:
for idx in range(len(xs)):
    print(idx, fnames[idx], DSCs[idx], HDs[idx])
    visualize(
        x=xs[idx][0], 
        y1=ys[idx][0][0],
        y2=ys[idx][0][1],
        y3=ys[idx][0][2],
        y4=ys[idx][0][3],
        y5=ys[idx][0][4], 
    )
    visualize(
        x=xs[idx][0], 
        yhat1 =yhats[idx][0][0].round(),
        yhat2 =yhats[idx][0][1].round(),
        yhat3 =yhats[idx][0][2].round(),
        yhat4 =yhats[idx][0][3].round(),
        yhat5 =yhats[idx][0][4].round(),
    )

    visualize(
        x=xs[idx][0], 
        yhat1_contour = mask2contour(yhats[idx][0][0].round()),
        yhat2_contour = mask2contour(yhats[idx][0][1].round()),
        yhat3_contour = mask2contour(yhats[idx][0][2].round()),
        yhat4_contour = mask2contour(yhats[idx][0][3].round()),
        yhat5_contour = mask2contour(yhats[idx][0][4].round()),
    )

In [ ]:
import natsort
import glob
import pydicom
import nibabel as nib
import numpy as np
import pylab as plt

dcms = natsort.natsorted(glob.glob('4th/DCM_test/*.dcm'))
labels = natsort.natsorted(glob.glob('4th/LABEL_test/*_5.nii.gz'))
len(dcms),len(labels)

In [ ]:
plt.hist(dcm.flatten(),bins=1024)

In [ ]:
# without
plt.hist(dcm.flatten(), bins=1024, cumulative=True, density=True)

In [ ]:
# with clipping
plt.hist(dcm.flatten(), bins=1024, cumulative=True, density=True)

In [ ]:
from skimage.morphology import binary_dilation, disk
for idx in range(1):
    dcm = pydicom.dcmread(dcms[idx])
    dcm = dcm.pixel_array
    dcm = dcm[300:]
    label = nib.load(labels[idx])
    label = label.get_fdata()
    label = label[...,0]
    label = np.moveaxis(label,0,1)
    label = label[300:]
    plt.figure(figsize=(20,12))
    plt.imshow(dcm,cmap='gray')
#     plt.imshow(binary_dilation(label,disk(4)),alpha=0.3)#,cmap='gray')
    plt.axis('off')
    plt.show()
    fname = dcms[idx].split('/')[-1][:-4]+'_5.png'
#     print(fname)
#     plt.savefig(fname)

In [ ]:
import pandas as pd
df = pd.read_csv('scoliosis_downtask_result.csv')

from sklearn.metrics import *
def metrics(yhat,y):
    """
    Binary classification metric
    
    input : long type inputs torch or numpy
    output : various metric in dictionary form
    """
    
    try:
        try:
            yhat = yhat.flatten().cpu().detach().numpy()
            y = y.flatten().cpu().detach().numpy()
        except:
            yhat = yhat.flatten().numpy()
            y = y.flatten().numpy()
    except:
        yhat = yhat.flatten()
        y = y.flatten()
    
    tn, fp, fn, tp = confusion_matrix(y, yhat).ravel()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    iou = tp/(tp+fp+fn)
    dice = 2*tp/(2*tp+fp+fn)
    specificity = tn / (tn+fp)
    sensitivity = tp / (tp+fn)
    ppv = tp / (tp+fp)
    npv = tn / (tn+fn)
    
#     df = pd.DataFrame([accuracy,dice,iou,npv,sensitivity,specificity,ppv],['accuracy','dice','iou','npv','sensitivity','specificity','ppv'])
#     return df
    return {'accuracy':accuracy,
            'dice':dice, 
            'iou':iou, 
            'npv':npv,
            'sensitivity':sensitivity,
            'specificity':specificity,
            'ppv':ppv,
           }

In [ ]:
thresholds = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
for threshold in thresholds: 
    y = df['y'].to_numpy()
    yhat = df['yhat'].to_numpy()

    yhat_ = yhat.copy()
    yhat_[yhat<threshold] = 0
    yhat_[yhat>=threshold] = 1
    print(threshold,metrics(yhat_,y))